# Main Notebook

This notebook contains the most relevant mech interp tools I've developed, centered around doing circuit analysis for any task

It took me longer than 15 hours to develop all this! However I picked a fresh task (copy) I'd never studied on mamba before, and the 15 hours were spent copying code from my other notebooks into here/cleaning it up/analyzing the results/training some extra SAEs.

## Imports/General Utility Functions

In [1]:
# Requires install of mamba lens
# https://github.com/Phylliida/MambaLens
# pip install git+https://github.com/Phylliida/MambaLens.git
# and also my implementation of ACDC (used for dataset managment)
# pip install git+https://github.com/Phylliida/ACDC.git

In [1]:

import torch
import pandas as pd
import plotly.express as px
import transformer_lens.utils as utils
from mamba_lens import HookedMamba
from acdc.data.utils import generate_dataset
from tqdm import tqdm
from acdc import get_pad_token
from acdc import accuracy_metric
from transformer_lens.hook_points import HookPoint
from functools import partial
from jaxtyping import Float
from IPython.display import clear_output, display
import ipywidgets
from mamba_lens.input_dependent_hooks import clean_hooks
from einops import rearrange
from collections import defaultdict
from dataclasses import dataclass, field
import traceback
from IPython.display import display, clear_output
from ipywidgets import widgets
from IPython.display import display, HTML
import pickle

from acdc import (
    Edge,
    ACDCConfig,
    LOG_LEVEL_INFO,
    ACDCEvalData,
)

from acdc import ACDCEvalData
from acdc import get_pad_token
from acdc import accuracy_metric

torch.set_grad_enabled(False)

# from transformer lens
def get_device():
    if torch.cuda.is_available():
        return torch.device("cuda")
    if torch.backends.mps.is_available() and torch.backends.mps.is_built():
        # Parse the PyTorch version to check if it's below version 2.0
        major_version = int(torch.__version__.split(".")[0])
        if major_version >= 2:
            return torch.device("mps")

    return torch.device("cpu")

# modified from neel nanda's examples
def imshow(tensor, renderer=None, xaxis="", yaxis="", font_size=None, show=True, color_continuous_midpoint=0.0, fix_size=False, height=None, **kwargs):
    fig = px.imshow(utils.to_numpy(tensor), color_continuous_midpoint=color_continuous_midpoint, color_continuous_scale="RdBu", labels={"x":xaxis, "y":yaxis}, **kwargs)
    if not font_size is None:
        if 'x' in kwargs:
            fig.update_layout(
              xaxis = dict(
                tickmode='array',
                tickvals = kwargs['x'],
                ticktext = kwargs['x'], 
                ),
               font=dict(size=font_size, color="black"))
        if 'y' in kwargs:
            fig.update_layout(
              yaxis = dict(
                tickmode='array',
                tickvals = kwargs['y'],
                ticktext = kwargs['y'], 
                ),
               font=dict(size=font_size, color="black"))
    if fix_size:
        # default settings aren't very good, these are better
        plot_args = {
            'width': 800,
            'height': 600,
            "autosize": False,
            'showlegend': True,
            'margin': {"l":0,"r":0,"t":100,"b":0}
        }
        if model.cfg.n_layers < len(kwargs['y']):
            plot_args['height'] *= model.cfg.D_conv
        
        if not height is None:
            plot_args['height'] = height
        fig.update_layout(**plot_args)
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ))
    if show:
        fig.show(renderer)
    else:
        return fig

def line(tensor, renderer=None, xaxis="", yaxis="", **kwargs):
    px.line(utils.to_numpy(tensor), labels={"x":xaxis, "y":yaxis}, **kwargs).show(renderer)

def scatter(x, y, xaxis="", yaxis="", caxis="", renderer=None, **kwargs):
    x = utils.to_numpy(x)
    y = utils.to_numpy(y)
    px.scatter(y=y, x=x, labels={"x":xaxis, "y":yaxis, "color":caxis}, **kwargs).show(renderer)

def bar_chart(data, x_labels, y_label, title, font_size=None):
    # it requires a pandas dict with the columns and rows named, annoying
    # by default rows and columns are named with ints so we relabel them accordingly
    renames = dict([(i, x_labels[i]) for i in range(len(x_labels))])
    ps = pd.DataFrame(data.cpu().numpy()).rename(renames, axis='rows').rename({0: y_label}, axis='columns')
    fig = px.bar(ps, y=y_label, x=x_labels, title=title)
    if not font_size is None:
        fig.update_layout(
          xaxis = dict(
            tickmode='array',
            tickvals = x_labels,
            ticktext = x_labels, 
            ),
           font=dict(size=font_size, color="black"))
    fig.show()

def get_batched_index_into(indices):
    '''
    given data that is [B,N,V] and indicies that are [B,N,K] with each index being an index into the V space
    this gives you indexes you can use to access your values
    '''
    first_axis = []
    second_axis = []
    third_axis = []
    B, _, _ = indices.size()
    for b in range(B):
        second, third = get_index_into(indices[b])
        first_axis.append(torch.full(second.size(), fill_value=b, device=model.cfg.device))
        second_axis.append(second)
        third_axis.append(third)

    return torch.cat(first_axis), torch.cat(second_axis), torch.cat(third_axis)

def get_index_into(indices):
    '''
    given data that is [N,V] and indicies that are [N,K] with each index being an index into the V space
    this gives you indexes you can use to access your values
    '''
    num_data, num_per_data = indices.size()
    # we want
    # [0,0,0,...,] num per data of these
    # [1,1,1,...,] num per data of these
    # ...
    # [num_data-1, num_data-1, ...]
    first_axis_index = torch.arange(num_data, dtype=torch.long).view(num_data, 1)*torch.ones([num_data, num_per_data], dtype=torch.long)
    # now we flatten it so it has an index for each term aligned with our indices
    first_axis_index = first_axis_index.flatten()
    second_axis_index = indices.flatten()
    return first_axis_index, second_axis_index

## Load Model

In [2]:

torch.set_grad_enabled(False)
device = get_device()
print("device", device)
model = HookedMamba.from_pretrained("state-spaces/mamba-370m", device=device)

device cuda


/opt/conda/lib/python3.10/site-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Moving model to device:  cuda


## Setup Data

In [3]:
def decode_and_encode(tokenizer, tokens):
    '''
    Gets rid of weird encoding issues by encoding and decoding
    The tokens will be different that's okay and intentional
    '''
    prompt = tokenizer.decode(tokens).encode("ascii", "ignore").decode("ascii", "ignore")
    return tokenizer.encode(prompt)

def copy_data_generator(tokenizer, num_patching_pairs, copy_seq_len, num_repeats):
    '''
    Generates copy_seq_len random tokens, repeated twice (with the last token cut off)
    This is just a test to see if it can copy the repeated sequence from before

    for example, 

    uncorrupted:
    a b c d a b c (answer is d)
    corrupted:
    a b c e a b c (answer is e)
    '''
    first_len = None

    # ignore special tokens like BOS or PAD
    special_token_ids = set()
    for special_token_name, token_str in model.tokenizer.special_tokens_map.items():
        special_token_ids.add(model.tokenizer.convert_tokens_to_ids([token_str])[0])
    valid_ids = []
    for tok in range(tokenizer.vocab_size):
        if not tok in special_token_ids:
            valid_ids.append(tok)    
    valid_ids = torch.tensor(valid_ids)
    for i in list(range(num_patching_pairs)):
        while True:
            # sample without replacement
            # one extra so corrupted isn't in sequence
            data = valid_ids[torch.randperm(len(valid_ids))[:copy_seq_len+1]].flatten()
            corrupted_id = data[-1]
            data_repeating = data[:-1]
            uncorrupted_data = torch.concatenate([data_repeating]*num_repeats + [data_repeating[:-1]])
            corrupted_data = torch.concatenate([data_repeating[:-1], torch.tensor([corrupted_id])]*num_repeats + [data_repeating[:-1]])
            # make sure it encodes and decodes properly
            uncorrupted_prompt = tokenizer.decode(uncorrupted_data).encode("ascii", "ignore").decode("ascii", "ignore")
            corrupted_prompt = tokenizer.decode(corrupted_data).encode("ascii", "ignore").decode("ascii", "ignore")
            uncorrupted_answer = tokenizer.decode(data_repeating[-1]).encode("ascii", "ignore").decode("ascii", "ignore")
            corrupted_answer = tokenizer.decode(torch.tensor([corrupted_id])).encode("ascii", "ignore").decode("ascii", "ignore")
            reencoded_uncorrupted_data = torch.tensor(tokenizer.encode(uncorrupted_prompt))
            reencoded_corrupted_data = torch.tensor(tokenizer.encode(corrupted_prompt))
            if reencoded_uncorrupted_data.size() == uncorrupted_data.size() and reencoded_corrupted_data.size() == corrupted_data.size() and torch.all(reencoded_uncorrupted_data == uncorrupted_data) and torch.all(reencoded_corrupted_data == corrupted_data):
                break
        yield uncorrupted_prompt, [uncorrupted_answer], [corrupted_answer]
        yield corrupted_prompt, [corrupted_answer], [uncorrupted_answer]

num_patching_pairs = 200
seed = 51
valid_seed = 51
constrain_to_answers = False
has_symmetric_patching = True
varying_data_lengths = False
copy_seq_len = 6
num_repeats = 2 # just immediately copying seems only somewhat doable by this model

data = generate_dataset(model=model,
              data_generator=copy_data_generator,
              num_patching_pairs=num_patching_pairs,
              seed=seed,
              valid_seed=valid_seed,
              constrain_to_answers=constrain_to_answers,
              has_symmetric_patching=has_symmetric_patching, 
              varying_data_lengths=varying_data_lengths,
              copy_seq_len=copy_seq_len,
              num_repeats=num_repeats)

for i in list(range(10)):
    uncorrupted_i = i*2
    corrupted_i = i*2+1
    uncorrupted = data.data[uncorrupted_i][:data.last_token_position[uncorrupted_i]+1]
    corrupted = data.data[corrupted_i][:data.last_token_position[corrupted_i]+1]
    print(i)
    print(model.to_str_tokens(uncorrupted))
    print(f"answers   are {model.to_str_tokens(data.correct[uncorrupted_i])}")
    print(f"incorrect are {model.to_str_tokens(data.incorrect[uncorrupted_i])}")
    print(model.to_str_tokens(corrupted))
    print(f"answers   are {model.to_str_tokens(data.correct[corrupted_i])}")
    print(f"incorrect are {model.to_str_tokens(data.incorrect[corrupted_i])}")

0
['<|endoftext|>', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' proteins', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' proteins', ' Electron', 'rising', ' inaccur', ' invisible', ',...,']
answers   are [' proteins']
incorrect are [' Love']
['<|endoftext|>', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' Love', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' Love', ' Electron', 'rising', ' inaccur', ' invisible', ',...,']
answers   are [' Love']
incorrect are [' proteins']
1
['<|endoftext|>', ' appreciation', ' swarm', ' uncertainties', ' register', ' 1985', ' persists', ' appreciation', ' swarm', ' uncertainties', ' register', ' 1985', ' persists', ' appreciation', ' swarm', ' uncertainties', ' register', ' 1985']
answers   are [' persists']
incorrect are ['633']
['<|endoftext|>', ' appreciation', ' swarm', ' uncertainties', ' register', ' 1985', '633', ' appreciation', ' swarm', ' uncertainties', ' register', ' 1985', '633', ' a

## Eval Data

In [ ]:

print("printing example data points:")
for b in range(4):
    pad_token = get_pad_token(model.tokenizer)
    # because there is padding if lengths vary, this only fetches the tokens that are part of the sequence
    toks = data.data[b][:data.last_token_position[b]+1]
    print(model.tokenizer.decode(toks))
    for ind, tok in enumerate(data.correct[b]):
        if tok != pad_token:
            print(f"  correct answer: {repr(model.tokenizer.decode([tok.item()]))}")
    for ind, tok in enumerate(data.incorrect[b]):
        if tok != pad_token:
            print(f"  incorrect answer: {repr(model.tokenizer.decode([tok.item()]))}")

TOP_K = 5
def logging_incorrect_metric(data: ACDCEvalData):
    pad_token = get_pad_token(model.tokenizer)
    for data_subset in [data.patched, data.corrupted]:
        batch, _ = data_subset.data.size()
        for b in range(batch):
            if not data_subset.top_is_correct[b].item():
                if not data.constrain_to_answers:
                    logits = data_subset.logits[b]
                    prs = torch.nn.functional.softmax(logits, dim=0)
                    top = torch.argsort(-logits)
                toks = data_subset.data[b][:data_subset.last_token_position[b]+1]
                print("failed on this data point:")
                print(model.to_str_tokens(toks))
                print("correct prs:")
                for i, tok in enumerate(data_subset.correct[b]):
                    if tok.item() != pad_token:
                        print(data_subset.correct_prs[b,i].item(), model.tokenizer.decode([tok.item()]))
                        if not data.constrain_to_answers:
                            top_k_pos = (top==tok.item()).nonzero().item()
                            print(f" top k pos of {top_k_pos}")
                print("incorrect prs:")
                for i, tok in enumerate(data_subset.incorrect[b]):
                    if tok.item() != pad_token:
                        print(data_subset.incorrect_prs[b,i].item(), model.tokenizer.decode([tok.item()]))
                        if not data.constrain_to_answers:
                            top_k_pos = (top==tok.item()).nonzero().item()
                            print(f" top k pos of {top_k_pos}")
                if not data.constrain_to_answers:
                    for i, tok in enumerate(top[:TOP_K]):
                        if tok.item() in [x.item() for x in data_subset.correct[b]]:
                            print(f"  correct   top {i} token {tok} = {repr(model.tokenizer.decode([tok]))} logit {logits[tok]} prs {prs[tok]}")
                        elif tok.item() in [x.item() for x in data_subset.incorrect[b]]:
                            print(f"  incorrect top {i} token {tok} = {repr(model.tokenizer.decode([tok]))} logit {logits[tok]} prs {prs[tok]}")
                        else:
                            print(f"  other     top {i} token {tok} = {repr(model.tokenizer.decode([tok]))} logit {logits[tok]} prs {prs[tok]}")
    return data.patched.correct_prs[:,0]

pr_correct = data.eval(model=model, batch_size=10, metric=logging_incorrect_metric)
print(pr_correct)
print(torch.mean(pr_correct))
accuracy = data.eval(model=model, batch_size=10, metric=accuracy_metric)
print(accuracy)
print(torch.mean(accuracy))

# Grid Search for hparams of dataset to get good enough accuracy that we have some leeway for circuit analysis (also, is it sensitive? Seems no)

In [ ]:

num_patching_pairs = 100
seed = 42
valid_seed = 41
constrain_to_answers = False
has_symmetric_patching = True
varying_data_lengths = False

max_seq_len = 40
max_num_repeats = 10

output_data = torch.zeros([max_seq_len-1, max_num_repeats-1])
for i, copy_seq_len in enumerate(range(1,max_seq_len)):
    print(copy_seq_len)
    for j, num_repeats in enumerate(tqdm(range(1,max_num_repeats))):
        seed = i+j*num_repeats
        data = generate_dataset(model=model,
                    data_generator=copy_data_generator,
                    num_patching_pairs=num_patching_pairs,
                    seed=seed,
                    valid_seed=valid_seed,
                    constrain_to_answers=constrain_to_answers,
                    has_symmetric_patching=has_symmetric_patching, 
                    varying_data_lengths=varying_data_lengths,
                    copy_seq_len=copy_seq_len,
                    num_repeats=num_repeats)
        output_data[i,j] = torch.mean(data.eval(model=model, batch_size=20, metric=accuracy_metric)).item()


In [ ]:
imshow(output_data.T, x=[str(x) for x in range(1,max_seq_len)], y=[str(y) for y in range(1,max_num_repeats)], fix_size=True, font_size=9, title='accuracy of mamba-370m on copy task', xaxis='num tokens in sequence', yaxis='num times repeated')

# here we see that it is very good as long as sequence longer than 3 and it's repeated at least twice. It's alright for repeated once but I'd prefer starting with a higher accuracy.
# I'll do 6 tokens in sequence, and repeat it 2 times (so 3 times total)
# 1 hr 5 min

## Patching

In [ ]:
prompt_uncorrupted = model.tokenizer.decode(data.data[0][1:])
prompt_corrupted = model.tokenizer.decode(data.data[1][1:])
uncorrupted_answer = model.tokenizer.decode(data.correct[0])
corrupted_answer = model.tokenizer.decode(data.correct[1])

print(prompt_uncorrupted)
print(prompt_corrupted)
print(uncorrupted_answer)
print(corrupted_answer)

In [ ]:
# modified from neel nanda's examples

H_N_PATCHING_LAYER = 39

# default settings aren't very good, these are better
plot_args = {
    'width': 800,
    'height': 600,
    "autosize": False,
    'showlegend': True,
    'margin': {"l":0,"r":0,"t":100,"b":0}
}

# you can modify this to only run things on a subset of layers
limited_layers = list(range(model.cfg.n_layers))


answer_tokens = sorted(list(set([model.tokenizer.encode(uncorrupted_answer)[0], model.tokenizer.encode(corrupted_answer)[0]])))

prompt_uncorrupted_tokens = model.to_tokens(prompt_uncorrupted)
prompt_corrupted_tokens = model.to_tokens(prompt_corrupted)

# logits should be [B,L,V] 
def uncorrupted_logit_minus_corrupted_logit(logits, uncorrupted_answer, corrupted_answer):
    uncorrupted_index = model.to_single_token(uncorrupted_answer)
    corrupted_index = model.to_single_token(corrupted_answer)
    return logits[0, -1, uncorrupted_index] - logits[0, -1, corrupted_index]

# prs should be [B,L,V] 
def uncorrupted_pr_minus_corrupted_pr(prs, uncorrupted_answer, corrupted_answer):
    uncorrupted_index = model.to_single_token(uncorrupted_answer)
    corrupted_index = model.to_single_token(corrupted_answer)
    return prs[0, -1, uncorrupted_index] - prs[0, -1, corrupted_index]



# [B,L,V]
corrupted_logits, corrupted_activations = model.run_with_cache(prompt_corrupted_tokens, only_use_these_layers=limited_layers)
corrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=corrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
corrupted_prs = torch.softmax(corrupted_logits, dim=2)
corrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=corrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)


# [B,L,V]
uncorrupted_logits = model.run_with_hooks(prompt_uncorrupted_tokens, only_use_these_layers=limited_layers)
uncorrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=uncorrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
uncorrupted_prs = torch.softmax(uncorrupted_logits, dim=2)
uncorrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=uncorrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)

uncorrupted_index = model.to_single_token(uncorrupted_answer)
corrupted_index = model.to_single_token(corrupted_answer)
print(f'uncorrupted prompt\n{prompt_uncorrupted}')
print(f"{repr(uncorrupted_answer)} logit {uncorrupted_logits[0,-1,uncorrupted_index]}")
print(f"{repr(uncorrupted_answer)} pr {uncorrupted_prs[0,-1,uncorrupted_index]}")
print(f"{repr(corrupted_answer)} logit {uncorrupted_logits[0,-1,corrupted_index]}")
print(f"{repr(corrupted_answer)} pr {uncorrupted_prs[0,-1,corrupted_index]}")
print(f'\ncorrupted prompt\n{prompt_corrupted}')
print(f"{repr(uncorrupted_answer)} logit {corrupted_logits[0,-1,uncorrupted_index]}")
print(f"{repr(uncorrupted_answer)} pr {corrupted_prs[0,-1,uncorrupted_index]}")
print(f"{repr(corrupted_answer)} logit {corrupted_logits[0,-1,corrupted_index]}")
print(f"{repr(corrupted_answer)} pr {corrupted_prs[0,-1,corrupted_index]}")

# We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
L = len(prompt_uncorrupted_tokens[0])
if len(prompt_corrupted_tokens[0]) != len(prompt_uncorrupted_tokens[0]):
    raise Exception("Prompts are not the same length") # feel free to comment this out, you can patch for different sized prompts its just a lil sus

# diff is logit of uncorrupted_answer - logit of corrupted_answer
# we expect corrupted_diff to have a negative value (as corrupted should put high pr on corrupted_answer)
# we expect uncorrupted to have a positive value (as uncorrupted should put high pr on uncorrupted_answer)
# thus we can treat these as (rough) min and max possible values
min_logit_diff = corrupted_logit_diff
max_logit_diff = uncorrupted_logit_diff

min_pr_diff = corrupted_pr_diff
max_pr_diff = uncorrupted_pr_diff



# make token labels that describe the patch
corrupted_str_tokens = model.to_str_tokens(prompt_corrupted_tokens)
uncorrupted_str_tokens = model.to_str_tokens(prompt_uncorrupted_tokens)
token_labels = []
for index, (corrupted_token, uncorrupted_token) in enumerate(zip(corrupted_str_tokens, uncorrupted_str_tokens)):
    if corrupted_token == uncorrupted_token:
        token_labels.append(f"{corrupted_token}_{index}")
    else:
        token_labels.append(f"{uncorrupted_token}->{corrupted_token}_{index}")

def run_patching(patching_type,
                 patching_hook_name_func,
                 patching_hook_func,
                 batch_size,
                 show_options, 
                 min_logit_diff,
                 max_logit_diff,
                 min_pr_diff,
                 max_pr_diff,
                 token_labels,
                 prompt_uncorrupted_tokens,
                 uncorrupted_answer,
                 corrupted_answer,
                 always_hooks=None,
                 show_plot=True,
                 **kwargs):
    _, L = prompt_uncorrupted_tokens.size()
    torch.cuda.empty_cache()
    hook_title = patching_hook_name_func(layer='{layer}', position='{position}')
    print(f"running patching {patching_type}, using hook {hook_title}")
    global patching_result_logits, patching_result_prs # if you want to access it once this is done running
    n_layers = len(limited_layers)

    num_results = n_layers
    if patching_type == H_N_PATCHING:
        print(f"on layer H_N_PATCHING_LAYER={H_N_PATCHING_LAYER}")
        N = model.cfg.N
        num_results = N
    elif patching_type == CONV_FILTERS_PATCHING:
        D_conv = model.cfg.D_conv
        num_results = (D_conv-1)*n_layers # -1 because the zero one is always zero so we ignore it
    
    patching_result_normalized_logits = torch.zeros((num_results, L), device=model.cfg.device)
    patching_result_normalized_prs = torch.zeros((num_results, L), device=model.cfg.device)

    num_answers = len(answer_tokens)
    patching_result_logits = torch.zeros((num_results, L, num_answers), device=model.cfg.device)
    patching_result_prs = torch.zeros((num_results, L, num_answers), device=model.cfg.device)
    
    hooks = []
    # skipping h needs A_bar stored, so also add that hook
    if patching_type == SKIPPING_H_PATCHING:
        for i, layer in list(enumerate(limited_layers)):
            hooks.append((f'blocks.{layer}.hook_A_bar', partial(A_bar_storage_hook_for_skipping_h, layer=layer)))

    # skipping layer needs layer_input (resid_pre) stored, so also add that hook
    if patching_type == LAYER_SKIPPING:
        for i, layer in list(enumerate(limited_layers)):
            hooks.append((f'blocks.{layer}.hook_resid_pre', partial(layer_input_storage_hook, layer=layer)))
    
    # conv filters works via initializing things, then storing all the stuff we want to hook, then doing all that in place at the same time
    if patching_type == CONV_FILTERS_PATCHING:
        for i, layer in list(enumerate(limited_layers)):
            # reset the storage to empty/initialize stuff
            hooks.append((f'blocks.{layer}.hook_layer_input', better_conv_patching_init_hook))
            # doing all the patches we have stored (below) at the same time
            hooks.append((f'blocks.{layer}.hook_conv', partial(better_conv_patching_hook, input_hook_name=f'blocks.{layer}.hook_in_proj', layer=layer)))

    if not always_hooks is None:
        hooks += always_hooks

    initial_num_hooks = len(hooks)

    
    if patching_type == H_N_PATCHING:
        batch = 0
        indices = []
        for n in range(N):
            for position in range(L):
                patching_hook_name = patching_hook_name_func(layer=H_N_PATCHING_LAYER, position=position)
                if batch_size != BATCH_SIZE_ALL: batch = batch % int(batch_size)
                patching_hook = partial(patching_hook_func, layer=H_N_PATCHING_LAYER, position=position, n=n, batch=batch)
                batch += 1
                indices.append((n,position))
                hooks.append((patching_hook_name, patching_hook))
    elif patching_type == CONV_FILTERS_PATCHING:
        batch = 0
        indices = []
        D_conv = model.cfg.D_conv
        ind = 0
        for i, layer in list(enumerate(limited_layers)):
            for conv_filter_i in range(D_conv):
                if conv_filter_i == 0: continue # this -d_conv-1 filter is always zero for some reason
                for position in range(L):
                    patching_hook_name = patching_hook_name_func(layer=layer, position=position)
                    if batch_size != BATCH_SIZE_ALL: batch = batch % int(batch_size)
                    hooks.append((f'blocks.{layer}.hook_in_proj', partial(better_conv_patching_storage_hook, position=position, layer=layer, conv_filter_i=conv_filter_i, batch=batch)))
                    #patching_hook = partial(patching_hook_func, layer=layer, position=position, batch=batch, conv_filter_i=conv_filter_i)
                    batch += 1
                    indices.append((ind,position))
                    #hooks.append((patching_hook_name, patching_hook))
                ind += 1
    else:
        batch = 0
        indices = []
        for i, layer in list(enumerate(limited_layers)):
            for position in range(L):
                patching_hook_name = patching_hook_name_func(layer=layer, position=position)
                if batch_size != BATCH_SIZE_ALL: batch = batch % int(batch_size)
                patching_hook = partial(patching_hook_func, layer=layer, position=position, batch=batch)
                batch += 1
                indices.append((i,position))
                hooks.append((patching_hook_name, patching_hook))

    
    if batch_size != BATCH_SIZE_ALL:
        V = model.cfg.V
        patched_logits = torch.zeros([len(indices), L, V])
        for batch_start in tqdm(list(range(0, len(indices), int(batch_size)))):
            batch_end = min(len(indices), batch_start+int(batch_size))
            # always do the initial they are for storage
            batch_hooks = hooks[:initial_num_hooks] + hooks[initial_num_hooks+batch_start:initial_num_hooks+batch_end]
            cur_batch_size = batch_end-batch_start
            patched_logits[batch_start:batch_end] = model.run_with_hooks(prompt_uncorrupted_tokens.expand(cur_batch_size,L), fwd_hooks=batch_hooks, only_use_these_layers=limited_layers, **kwargs)
    else:
        # [B,L,V]
        patched_logits = model.run_with_hooks(prompt_uncorrupted_tokens.expand(batch,L), fwd_hooks=hooks, only_use_these_layers=limited_layers, **kwargs)
   
    # [B,L,V]
    patched_prs = torch.softmax(patched_logits, dim=2)
    print("finished patching, plotting...")
    for b, (i,position) in enumerate(indices):
        if corrupted_answer != uncorrupted_answer:
            patched_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=patched_logits[b:b+1],
                                                                         uncorrupted_answer=uncorrupted_answer,
                                                                         corrupted_answer=corrupted_answer)
            # normalize it so
            # 0 means min_logit_diff (so 0 means that it is acting like the corrupted model)
            # 1 means max_logit_diff (so 1 means that it is acting like the uncorrupted model)
            normalized_patched_logit_diff = (patched_logit_diff-min_logit_diff)/(max_logit_diff - min_logit_diff)
            # now flip them, since most interventions will do nothing and thus act like uncorrupted model, visually its better to have that at 0
            # so now
            # 0 means that it is acting like the uncorrupted model
            # 1 means that it is acting like the corrupted model
            normalized_patched_logit_diff = 1.0 - normalized_patched_logit_diff
            normalized_patched_logit_diff = normalized_patched_logit_diff #normalized_always_logit_diff - normalized_patched_logit_diff
            patching_result_normalized_logits[i, position] = normalized_patched_logit_diff
            
            # same for pr
            patched_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=patched_prs[b:b+1],
                                                                uncorrupted_answer=uncorrupted_answer,
                                                                corrupted_answer=corrupted_answer)
            normalized_patched_pr_diff = 1.0-(patched_pr_diff-min_pr_diff)/(max_pr_diff - min_pr_diff)
            normalized_patched_pr_diff = normalized_always_pr_diff - normalized_patched_pr_diff
            patching_result_normalized_prs[i, position] = normalized_patched_pr_diff

        for k, answer_token in enumerate(answer_tokens):
            patching_result_logits[i, position, k] = patched_logits[b,-1,answer_token]
            patching_result_prs[i, position, k] = patched_prs[b,-1,answer_token]
    
        
    if patching_type == H_N_PATCHING:
        layer_labels = [str(n) for n in range(N)]
    elif patching_type == CONV_FILTERS_PATCHING:
        layer_labels = []
        for layer in limited_layers:
            for conv_i in range(1, D_conv):
                layer_labels.append(f"layer {layer} conv {conv_i-D_conv+1}")
    else:
        layer_labels = [str(layer) for layer in limited_layers]
    figs = []
    y_axis = 'Layer'
    if patching_type == H_N_PATCHING:
        y_axis = 'N'
    if show_plot:
        if corrupted_answer != uncorrupted_answer:
            if show_options in [SHOW_LOGITS, SHOW_BOTH]:
                figs.append(imshow(patching_result_normalized_logits, x=token_labels, y=layer_labels, xaxis="Position", yaxis=y_axis, title=f"Normalized logit difference after patching {patching_type} using hook {hook_title}", font_size=8, show=False))
            if show_options in [SHOW_PR, SHOW_BOTH]:
                figs.append(imshow(patching_result_normalized_prs, x=token_labels, y=layer_labels, xaxis="Position", yaxis=y_axis, title=f"Normalized pr difference after patching {patching_type} using hook {hook_title}", font_size=8, show=False))
        
        for k, answer_token in enumerate(answer_tokens):
            if show_options in [SHOW_LOGITS, SHOW_BOTH]:
                figs.append(imshow(patching_result_logits[:,:,k], color_continuous_midpoint=None, x=token_labels, y=layer_labels, xaxis="Position", yaxis="Layer", title=f"Logit of uncorrupted answer {repr(model.tokenizer.decode([answer_token]))} after patching {patching_type} using hook {hook_title}", font_size=8, show=False))
            if show_options in [SHOW_PR, SHOW_BOTH]:
                figs.append(imshow(patching_result_prs[:,:,k], x=token_labels, y=layer_labels, xaxis="Position", yaxis=y_axis, title=f"Pr of uncorrupted answer {repr(model.tokenizer.decode([answer_token]))} after patching {patching_type} using hook {hook_title}", font_size=8, show=False)) 
        
        for fig in figs:
            plot_args_copy = dict(list(plot_args.items()))
            if patching_type == CONV_FILTERS_PATCHING:
                plot_args_copy['height'] *= D_conv
            fig.update_layout(**plot_args_copy)
            fig.update_layout(legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            ))
            fig.show()
    else:
        return layer_labels, y_axis, patching_result_normalized_logits, patching_result_normalized_prs, patching_result_logits, patching_result_prs

## hooks for conv filter patching
def conv_input_storage_hook(
    conv_input: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    layer: int,
) -> Float[torch.Tensor, "B L E"]:
    global progress # it's slow enough that progress bar is useful
    if layer == 0:
        progress = tqdm(total=len(limited_layers))
    else:
        progress.update(1)
    global storage
    storage = {}
    storage['conv_input'] = conv_input
    return conv_input

def conv_patching_hook(
    conv_output: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    layer: int,
    position: int,
    batch: int,
    conv_filter_i: int,
) -> Float[torch.Tensor, "B L E"]:
    global storage
    conv_input = storage['conv_input']
    B, L, E = conv_input.size()
    conv_input = rearrange(conv_input, 'B L E -> B E L')
    conv_input_corrupted = rearrange(corrupted_activations[f'blocks.{layer}.hook_in_proj'], 'B L E -> B E L')
    
    ### This is identical to what the conv is doing
    # pad zeros in front
    # [B,E,D_CONV-1+L]
    D_CONV = model.cfg.d_conv
    padded_input = torch.nn.functional.pad(conv_input, (D_CONV-1,0), mode='constant', value=0)
    padded_input_corrupted = torch.nn.functional.pad(conv_input_corrupted, (D_CONV-1,0), mode='constant', value=0)
    output = torch.zeros([B,E,L], device=model.cfg.device)
    # [E,1,D_CONV]
    conv_weight = model.blocks[layer].conv1d.weight
    # [E]
    conv_bias = model.blocks[layer].conv1d.bias
    # this is inefficient because its recomputing things every time
    # but I don't want to have to rely on the ordering of hooks because that's sus
    # so this is good enough
    for i in range(D_CONV):
        filter_str = f'filter_{i}'
        if not filter_str in storage:
            # [B,E,L]                      [E,1]                      [B,E,L]
            filter_contribution = conv_weight[:,0,i].view(E,1)*padded_input[:,:,i:i+L]
            storage[filter_str] = filter_contribution
        filter_contribution = storage[filter_str]
        if i == conv_filter_i:
            # [1,E,L]                                   [E,1]                          # [1,E,L]
            corrupted_filter_contribution = conv_weight[:,0,i].view(E,1)*padded_input_corrupted[:,:,i:i+L]
            # [E]                                                    [E]
            filter_contribution[batch,:,position] = corrupted_filter_contribution[0,:,position]
        storage[filter_str] = filter_contribution
        output += filter_contribution
        #output += conv_weight[:,0,i].view(E,1)*conv_input
        #if i == D_CONV-1:
        #    output += conv_weight[:,0,i].view(E,1)*conv_input

    # bias is not dependent on input so no reason to patch on it, just apply it as normal
    output += conv_bias.view(E, 1)
    
    output = rearrange(output, 'B E L -> B L E')
    return output


# we do a hacky thing where this first hook clears the global storage
# second hook stores all the hooks
# then third hook computes the output (over all the hooks)
# this avoids recomputing and so is much faster
global storage
global conv_storage
storage = {}
conv_storage = {}
CONV_HOOKS = "conv hooks"
CONV_BATCHES = "conv batches"
def better_conv_patching_init_hook(
    x,
    hook: HookPoint,
    **kwargs
):
    #print("init hook with layer", hook.name)
    # we need to clear this here
    # i tried having a "current layer" variable in the conv_storage that only clears when it doesn't match
    # but that doesn't work if you only patch the same layer over and over,
    # as stuff gets carried over
    # this way of doing things is much safer and lets us assume it'll be empty
    # well not quite, note that conv_patching_hook will be called with different batch_start and batch_end inputs during one forward pass
    # so we need to account for that in the keys we use
    global conv_storage
    global storage
    storage = {}
    conv_storage = {CONV_BATCHES: set()}
    return x

def better_conv_patching_storage_hook(
    x,
    hook: HookPoint,
    conv_filter_i: int,
    position: int,
    layer: int,
    batch: int,
    **kwargs,
):
    #print("append hook with layer", hook.name, "conv filter", conv_filter_i, "position", position, "layer", layer, "batch", batch)
    batch_start = batch
    batch_end = batch+1
    global storage
    storage[hook.name] = x
    global conv_storage
    hooks_key = (CONV_HOOKS, batch_start, batch_end)
    if not hooks_key in conv_storage:
        conv_storage[hooks_key] = [] # we can't do this above because it'll be emptied again on the next batch before this is called
    conv_storage[hooks_key].append({"position": position, "conv_filter_i": conv_filter_i})
    conv_storage[CONV_BATCHES].add((batch_start, batch_end))
    return x

from jaxtyping import Float
from einops import rearrange
global corrupted_activations

global conv_storage
def better_conv_patching_hook(
    conv_output: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    input_hook_name: str,
    layer: int,
    **kwargs,
) -> Float[torch.Tensor, "B L E"]:
    global conv_storage
    global storage
    ### This is identical to what the conv is doing
    # but we break it apart so we can patch on individual filters
    
    D_CONV = model.cfg.d_conv

    global corrupted_activations
    # [E,1,D_CONV]
    conv_weight = model.blocks[layer].conv1d.weight
    # [E]
    conv_bias = model.blocks[layer].conv1d.bias
    
    # don't recompute these if we don't need to
    # because we stored all the hooks and batches in conv_storage, we can just do them all at once
    output_key = f'output' # they need to share an output because they write to the same output tensor
    if not output_key in conv_storage:
        #print("layer", layer, "keys", conv_storage)
        apply_to_all_hooks = [] # this is important because otherwise the [0:None] would overwrite the previous results (or vice versa)
        apply_to_all_key = (CONV_HOOKS, 0, None)
        if apply_to_all_key in conv_storage:
            apply_to_all_hooks = conv_storage[apply_to_all_key]
        for batch_start, batch_end in conv_storage[CONV_BATCHES]:
            if batch_start == 0 and batch_end == None: continue # we cover this in the apply to all hooks above
            def get_filter_key(i):
                return f'filter_{i}'
            conv_input_uncorrupted = storage[input_hook_name][batch_start:batch_end]
            conv_input_corrupted = corrupted_activations[input_hook_name]
            B, L, E = conv_input_uncorrupted.size()
            
            conv_input_uncorrupted = rearrange(conv_input_uncorrupted, 'B L E -> B E L')
            conv_input_corrupted = rearrange(conv_input_corrupted, 'B L E -> B E L')
            
            # pad zeros in front
            # [B,E,D_CONV-1+L]
            padded_input_uncorrupted = torch.nn.functional.pad(conv_input_uncorrupted, (D_CONV-1,0), mode='constant', value=0)
            padded_input_corrupted = torch.nn.functional.pad(conv_input_corrupted, (D_CONV-1,0), mode='constant', value=0)
    
            # compute the initial filter values
            for i in range(D_CONV):
                filter_key = get_filter_key(i)
                # [B,E,L]                      [E,1]                      [B,E,L]
                filter_contribution = conv_weight[:,0,i].view(E,1)*padded_input_uncorrupted[:,:,i:i+L]
                conv_storage[filter_key] = filter_contribution
            
            # apply all the hooks
            for hook in conv_storage[(CONV_HOOKS, batch_start, batch_end)] + apply_to_all_hooks:
                position = hook['position']
                conv_filter_i = hook['conv_filter_i']
                #print(f"position {position} conv_filter_i {conv_filter_i} batch_start {batch_start} batch_end {batch_end}")
                filter_key = get_filter_key(conv_filter_i)
                # [1,E,L]                                   [E,1]                          # [B,E,L]
                corrupted_filter_contribution = conv_weight[:,0,conv_filter_i].view(E,1)*padded_input_corrupted[:,:,conv_filter_i:conv_filter_i+L]
                filter_contribution = conv_storage[filter_key]
                if position is None:
                    # [B,E,L]                    [B,E,L]
                    filter_contribution = corrupted_filter_contribution
                else:
                    # [B,E]                                                  [B,E]
                    filter_contribution[:,:,position] = corrupted_filter_contribution[:,:,position]
                conv_storage[filter_key] = filter_contribution
            
            # compute the output
            output = torch.zeros([B,E,L], device=model.cfg.device)
            #print(f'B {B} B2 {B2} E {E} L {L} conv_storage keys {conv_storage.keys()} filter sizes {[(k,v.size()) for (k,v) in conv_storage.items() if not type(v) is int]}')
            for i in range(D_CONV):
                filter_key = get_filter_key(i)
                output += conv_storage[filter_key]
                del conv_storage[filter_key] # clean up now we are done with it, just to be safe
            # bias is not dependent on input so no reason to patch on it, just apply it as normal
            output += conv_bias.view(E, 1)
            output = rearrange(output, 'B E L -> B L E')
            # interleave it back with the corrupted as every other
            conv_output[batch_start:batch_end] = output
        conv_storage[output_key] = conv_output
    return conv_storage[output_key]
           




## hooks for layer skipping
def layer_input_storage_hook(
    layer_input: Float[torch.Tensor, "B L D"],
    hook: HookPoint,
    layer: int,
) -> Float[torch.Tensor, "B L D"]:
    global storage
    storage = {}
    storage['layer_input'] = layer_input
    return layer_input

def layer_output_skipping_hook(
    layer_output: Float[torch.Tensor, "B L D"],
    hook: HookPoint,
    position: int,
    layer: int,
    batch: int,
) -> Float[torch.Tensor, "B L D"]:
    global storage
    layer_input = storage['layer_input']
    # intervene on the batch at the position
    layer_output[batch,position,:] = layer_input[batch,position,:]
    return layer_output


## hooks for h skipping
def A_bar_storage_hook_for_skipping_h(
    A_bar: Float[torch.Tensor, "B L E N"],
    hook: HookPoint,
    layer: int,
) -> Float[torch.Tensor, "B L E N"]:
    global storage
    storage = {}
    storage['A_bar'] = A_bar
    return A_bar

def skipping_h_hook(
    h: Float[torch.Tensor, "B E N"],
    hook: HookPoint,
    position: int,
    layer: int,
    batch: int,
) -> Float[torch.Tensor, "B E N"]:
    #print("fetching", storage[grab_pos][0,0,0:5], "from position", grab_pos)
    #print("my value (being ignore) is", h[0,0,0:5])
    #print(f"skipping ahead h at position {position}")
    global storage
    B,E,N = h.size()
    grab_pos = position-1
    if grab_pos < 0:
        h[batch,:,:] = torch.zeros((E,N), device=model.cfg.device)
    else:
        B,E,N = h.size()
        A_contribution = torch.ones((E,N), device=model.cfg.device)
        for missed_pos in range(grab_pos+1, position+1):
            A_contribution *= storage['A_bar'][batch,missed_pos,:,:]
        h_stored = storage[grab_pos][batch,:,:]
        h[batch,:,:] = A_contribution*h_stored
        #return A_contribution*storage[grab_pos]
    storage[position] = h
    return h


## Regular patching hooks
def position_patching_hook( # also works for B L E, B L E N, and B L N sized things
    x: Float[torch.Tensor, "B L D"],
    hook: HookPoint,
    position: int,
    layer: int, # we don't care about this
    batch: int,
) -> Float[torch.Tensor, "B L D"]:
    # only intervene on the specific pos
    corrupted_x = corrupted_activations[hook.name]
    x[batch, position, :] = corrupted_x[0, position, :]
    return x

def h_patching_hook(
    h: Float[torch.Tensor, "B E N"],
    hook: HookPoint,
    position: int,
    layer: int,
    batch: int,
) -> Float[torch.Tensor, "B E N"]:
    corrupted_h = corrupted_activations[hook.name]
    h[batch] = corrupted_h[0]
    return h

def h_n_patching_hook(
    h: Float[torch.Tensor, "B E N"],
    hook: HookPoint,
    position: int,
    layer: int,
    n: int,
    batch: int,
) -> Float[torch.Tensor, "B E N"]:
    corrupted_h = corrupted_activations[hook.name]
    h[batch,:,n] = corrupted_h[0,:,n]
    return h




def generate_always_hooks():
    hooks = []

    LAYER = 39
    _, L = prompt_uncorrupted_tokens.size()
    #for pos in range(L):
        # slice(None, None) is same as : (which means all)
        #hooks.append((f'blocks.{LAYER}.hook_h.{pos}', partial(h_patching_hook, layer=LAYER, position=pos, batch=slice(None, None))))
    #    hooks.append((f'blocks.{LAYER}.hook_layer_input', partial(position_patching_hook, layer=LAYER, position=pos, batch=slice(None, None))))

    #ABLATE_POS = 3
    #hooks.append((f'blocks.{35}.hook_layer_input', partial(position_patching_hook, layer=LAYER, position=3, batch=slice(None, None))))
    #hooks.append((f'blocks.{40}.hook_layer_input', partial(position_patching_hook, layer=LAYER, position=3, batch=slice(None, None))))
    #FINAL_POS = 19
    #hooks.append((f'blocks.{47}.hook_layer_input', partial(position_patching_hook, layer=LAYER, position=FINAL_POS, batch=slice(None, None))))
    return hooks
always_hooks = generate_always_hooks()

always_logits = model.run_with_hooks(prompt_uncorrupted_tokens, fwd_hooks=always_hooks, only_use_these_layers=limited_layers, fast_ssm=False, fast_conv=False)


always_prs = torch.softmax(always_logits, dim=2)
always_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=always_logits,
                                                                uncorrupted_answer=uncorrupted_answer,
                                                                corrupted_answer=corrupted_answer)
# normalize it so
# 0 means min_logit_diff (so 0 means that it is acting like the corrupted model)
# 1 means max_logit_diff (so 1 means that it is acting like the uncorrupted model)
normalized_always_logit_diff = (always_logit_diff-min_logit_diff)/(max_logit_diff - min_logit_diff)
# now flip them, since most interventions will do nothing and thus act like uncorrupted model, visually its better to have that at 0
# so now
# 0 means that it is acting like the uncorrupted model
# 1 means that it is acting like the corrupted model
normalized_always_logit_diff = 1.0 - normalized_always_logit_diff

# same for pr
always_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=always_prs,
                                                    uncorrupted_answer=uncorrupted_answer,
                                                    corrupted_answer=corrupted_answer)
normalized_always_pr_diff = 1.0-(always_pr_diff-min_pr_diff)/(max_pr_diff - min_pr_diff)




SKIPPING_H_PATCHING = 'skipping h'
H_N_PATCHING = 'h_n'
LAYER_SKIPPING = 'skipping layer'
CONV_FILTERS_PATCHING = 'conv filters'

patching_types = {
    'resid pre': (lambda layer, position: f'blocks.{layer}.hook_resid_pre', position_patching_hook),
    'layer input': (lambda layer, position: f'blocks.{layer}.hook_layer_input', position_patching_hook),
    'normalized input': (lambda layer, position: f'blocks.{layer}.hook_normalized_input', position_patching_hook),
    'skip': (lambda layer, position: f'blocks.{layer}.hook_skip', position_patching_hook), 
    'in proj': (lambda layer, position: f'blocks.{layer}.hook_in_proj', position_patching_hook), 
    CONV_FILTERS_PATCHING: (lambda layer, position: f'blocks.{layer}.hook_conv', conv_patching_hook),
    'conv': (lambda layer, position: f'blocks.{layer}.hook_conv', position_patching_hook), 
    'delta 1': (lambda layer, position: f'blocks.{layer}.hook_delta_1', position_patching_hook), 
    'delta 2': (lambda layer, position: f'blocks.{layer}.hook_delta_2', position_patching_hook), 
    'delta': (lambda layer, position: f'blocks.{layer}.hook_delta', position_patching_hook), 
    'A_bar': (lambda layer, position: f'blocks.{layer}.hook_A_bar', position_patching_hook), 
    'B': (lambda layer, position: f'blocks.{layer}.hook_B', position_patching_hook), 
    'B_bar': (lambda layer, position: f'blocks.{layer}.hook_B_bar', position_patching_hook), 
    'C': (lambda layer, position: f'blocks.{layer}.hook_C', position_patching_hook), 
    'ssm input': (lambda layer, position: f'blocks.{layer}.hook_ssm_input', position_patching_hook),
    SKIPPING_H_PATCHING: (lambda layer, position: f'blocks.{layer}.hook_h.{position}', skipping_h_hook),
    'h': (lambda layer, position: f'blocks.{layer}.hook_h.{position}', h_patching_hook),
    H_N_PATCHING: (lambda layer, position: f'blocks.{layer}.hook_h.{position}', h_n_patching_hook),
    'y': (lambda layer, position: f'blocks.{layer}.hook_y', position_patching_hook),
    'ssm output': (lambda layer, position: f'blocks.{layer}.hook_ssm_output', position_patching_hook),
    'after skip': (lambda layer, position: f'blocks.{layer}.hook_after_skip', position_patching_hook),
    'out proj': (lambda layer, position: f'blocks.{layer}.hook_out_proj', position_patching_hook),
    'resid post': (lambda layer, position: f'blocks.{layer}.hook_resid_post', position_patching_hook),
    LAYER_SKIPPING: (lambda layer, position: f'blocks.{layer}.hook_resid_post', layer_output_skipping_hook),
}

patching_types_keys = list(patching_types.keys())

def choose_patching_type(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_patching_type.patching_type = change['new'] # hack, gives this function the patching_type attribute

choose_patching_type.patching_type = patching_types_keys[0]

patching_type_dropdown = ipywidgets.Dropdown(
    options=patching_types_keys,
    value=patching_types_keys[0],
    description='patching type',
)
patching_type_dropdown.observe(choose_patching_type)
display(patching_type_dropdown)

BATCH_SIZE_ALL = 'all'
batch_size_keys = [BATCH_SIZE_ALL] + [str(b) for b in range(model.cfg.n_layers*model.cfg.D_conv*L)]

def choose_batch_size(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_batch_size.batch_size = change['new']

choose_batch_size.batch_size = batch_size_keys[0]

choose_batch_size_dropdown = ipywidgets.Dropdown(
    options=batch_size_keys,
    value=batch_size_keys[0],
    description='batch size',
)
choose_batch_size_dropdown.observe(choose_batch_size)
display(choose_batch_size_dropdown)

fast_conv_keys = ['True', 'False']

def choose_fast_conv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_conv.fast_conv = change['new'] == 'True'

choose_fast_conv.fast_conv = fast_conv_keys[0] == 'True'

choose_fast_conv_dropdown = ipywidgets.Dropdown(
    options=fast_conv_keys,
    value=fast_conv_keys[0],
    description='fast conv',
)
choose_fast_conv_dropdown.observe(choose_fast_conv)
display(choose_fast_conv_dropdown)


fast_ssm_keys = ['False', 'True']

def choose_fast_ssm(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_ssm.fast_ssm = change['new'] == 'True'

choose_fast_ssm.fast_ssm = fast_ssm_keys[0] == 'True'

choose_fast_ssm_dropdown = ipywidgets.Dropdown(
    options=fast_ssm_keys,
    value=fast_ssm_keys[0],
    description='fast ssm',
)
choose_fast_ssm_dropdown.observe(choose_fast_ssm)
display(choose_fast_ssm_dropdown)

SHOW_PR = 'Pr'
SHOW_LOGITS = 'Logits'
SHOW_BOTH = 'Both'
show_options = [SHOW_LOGITS, SHOW_PR, SHOW_BOTH]

def choose_show_options(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_show_options.show_options = change['new']

choose_show_options.show_options = show_options[0]

show_options_dropdown = ipywidgets.Dropdown(
    options=show_options,
    value=show_options[0],
    description='logits or pr',
)
show_options_dropdown.observe(choose_show_options)
display(show_options_dropdown)



def do_patching(arg, show_plot=True):
    with output: # this lets the stuff we output here be visible
        clear_output()
        patching_type = choose_patching_type.patching_type
        hook_name_func, hook_func = patching_types[patching_type]
        return run_patching(
                     patching_type=patching_type,
                     patching_hook_name_func=hook_name_func,
                     patching_hook_func=hook_func,
                     batch_size=choose_batch_size.batch_size,
                     fast_ssm=choose_fast_ssm.fast_ssm,
                     fast_conv=choose_fast_conv.fast_conv,
                     show_options=choose_show_options.show_options,
                     show_plot=show_plot,
                     min_logit_diff=min_logit_diff,
                     max_logit_diff=max_logit_diff,
                     min_pr_diff=min_pr_diff,
                     max_pr_diff=max_pr_diff,
                     token_labels=token_labels,
                     prompt_uncorrupted_tokens=prompt_uncorrupted_tokens,
                     uncorrupted_answer=uncorrupted_answer,
                     corrupted_answer=corrupted_answer,
                     always_hooks=generate_always_hooks())

patching_button = ipywidgets.Button(description = 'Run Patching')
patching_button.on_click(do_patching)
display(patching_button)

# you can't just display stuff inside a widget callback, you need a wrap any display code in this
output = ipywidgets.Output()
display(output)

# Average Patchings over multiple data points

In [ ]:

prompt_uncorrupted = "".join(model.tokenizer.decode(data.data[0][1:]))
prompt_corrupted = "".join(model.tokenizer.decode(data.data[1][1:]))
# note the spaces are important due to tokenization
uncorrupted_answer = "".join(model.tokenizer.decode([data.correct[0][0]]))
corrupted_answer = "".join(model.tokenizer.decode([data.correct[1][0]]))

print(prompt_uncorrupted)
print(prompt_corrupted)
print(uncorrupted_answer)
print(corrupted_answer)

def choose_patching_type(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_patching_type.patching_type = change['new'] # hack, gives this function the patching_type attribute

choose_patching_type.patching_type = patching_types_keys[0]

patching_type_dropdown = ipywidgets.Dropdown(
    options=patching_types_keys,
    value=patching_types_keys[0],
    description='patching type',
)
patching_type_dropdown.observe(choose_patching_type)
display(patching_type_dropdown)

BATCH_SIZE_ALL = 'all'
batch_size_keys = [BATCH_SIZE_ALL] + [str(b) for b in range(model.cfg.n_layers*model.cfg.D_conv*L)]

def choose_batch_size(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_batch_size.batch_size = change['new']

choose_batch_size.batch_size = batch_size_keys[0]

choose_batch_size_dropdown = ipywidgets.Dropdown(
    options=batch_size_keys,
    value=batch_size_keys[0],
    description='batch size',
)
choose_batch_size_dropdown.observe(choose_batch_size)
display(choose_batch_size_dropdown)

fast_conv_keys = ['False', 'True']

def choose_fast_conv(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_conv.fast_conv = change['new'] == 'True'

choose_fast_conv.fast_conv = fast_conv_keys[0] == 'True'

choose_fast_conv_dropdown = ipywidgets.Dropdown(
    options=fast_conv_keys,
    value=fast_conv_keys[0],
    description='fast conv',
)
choose_fast_conv_dropdown.observe(choose_fast_conv)
display(choose_fast_conv_dropdown)


fast_ssm_keys = ['False', 'True']

def choose_fast_ssm(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_fast_ssm.fast_ssm = change['new'] == 'True'

choose_fast_ssm.fast_ssm = fast_ssm_keys[0] == 'True'

choose_fast_ssm_dropdown = ipywidgets.Dropdown(
    options=fast_ssm_keys,
    value=fast_ssm_keys[0],
    description='fast ssm',
)
choose_fast_ssm_dropdown.observe(choose_fast_ssm)
display(choose_fast_ssm_dropdown)

SHOW_PR = 'Pr'
SHOW_LOGITS = 'Logits'
SHOW_BOTH = 'Both'
show_options = [SHOW_LOGITS, SHOW_PR, SHOW_BOTH]

def choose_show_options(change):
    if change['type'] == 'change' and change['name'] == 'value':
        choose_show_options.show_options = change['new']

choose_show_options.show_options = show_options[0]

show_options_dropdown = ipywidgets.Dropdown(
    options=show_options,
    value=show_options[0],
    description='logits or pr',
)
show_options_dropdown.observe(choose_show_options)
display(show_options_dropdown)
H_N_PATCHING_LAYER = 39


global corrupted_activations

def do_patching(arg, show_plot=True):
    patching_result_normalized_logits_total = None
    with output:
        clear_output()
        iters = 0
        for i in range(data.data.size()[0]//2):
            uncorrupted_i = i*2
            corrupted_i = i*2+1
            uncorrupted = data.data[uncorrupted_i]
            corrupted = data.data[corrupted_i]
    
            uncorrupted_answer = model.tokenizer.decode([data.correct[uncorrupted_i][0]])
            corrupted_answer = model.tokenizer.decode([data.correct[corrupted_i][0]])
            answer_tokens = sorted(list(set([model.tokenizer.encode(uncorrupted_answer)[0], model.tokenizer.encode(corrupted_answer)[0]])))
        
            prompt_uncorrupted_tokens = uncorrupted[:data.last_token_position[uncorrupted_i]+1].view(1,-1)
            prompt_corrupted_tokens = corrupted[:data.last_token_position[corrupted_i]+1].view(1,-1)

            #print(model.to_str_tokens(prompt_uncorrupted_tokens))
            #print(model.to_str_tokens(prompt_corrupted_tokens))
            #print("answers")
            #print(uncorrupted_answer, corrupted_answer)
            # [B,L,V]
            global corrupted_activations
            corrupted_logits, corrupted_activations = model.run_with_cache(prompt_corrupted_tokens, only_use_these_layers=limited_layers)
            corrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=corrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
            corrupted_prs = torch.softmax(corrupted_logits, dim=2)
            corrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=corrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
            
            # [B,L,V]
            uncorrupted_logits = model(prompt_uncorrupted_tokens, only_use_these_layers=limited_layers)
            uncorrupted_logit_diff = uncorrupted_logit_minus_corrupted_logit(logits=uncorrupted_logits, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
            uncorrupted_prs = torch.softmax(uncorrupted_logits, dim=2)
            uncorrupted_pr_diff = uncorrupted_pr_minus_corrupted_pr(prs=uncorrupted_prs, uncorrupted_answer=uncorrupted_answer, corrupted_answer=corrupted_answer)
            
            uncorrupted_index = model.to_single_token(uncorrupted_answer)
            corrupted_index = model.to_single_token(corrupted_answer)
            
            # We make a tensor to store the results for each patching run. We put it on the model's device to avoid needing to move things between the GPU and CPU, which can be slow.
            L = len(prompt_uncorrupted_tokens[0])
            if len(prompt_corrupted_tokens[0]) != len(prompt_uncorrupted_tokens[0]):
                raise Exception("Prompts are not the same length") # feel free to comment this out, you can patch for different sized prompts its just a lil sus
        
            # diff is logit of uncorrupted_answer - logit of corrupted_answer
            # we expect corrupted_diff to have a negative value (as corrupted should put high pr on corrupted_answer)
            # we expect uncorrupted to have a positive value (as uncorrupted should put high pr on uncorrupted_answer)
            # thus we can treat these as (rough) min and max possible values
            min_logit_diff = corrupted_logit_diff
            max_logit_diff = uncorrupted_logit_diff
            
            min_pr_diff = corrupted_pr_diff
            max_pr_diff = uncorrupted_pr_diff
            
            # make token labels that describe the patch
            corrupted_str_tokens = model.to_str_tokens(prompt_corrupted_tokens)
            uncorrupted_str_tokens = model.to_str_tokens(prompt_uncorrupted_tokens)
            token_labels = []
            for index, (corrupted_token, uncorrupted_token) in enumerate(zip(corrupted_str_tokens, uncorrupted_str_tokens)):
                if corrupted_token == uncorrupted_token:
                    token_labels.append(f"{corrupted_token}_{index}")
                else:
                    token_labels.append(f"{uncorrupted_token}->{corrupted_token}_{index}")
        
                
            print(i, "/", data.data.size()[0]//2)
            patching_type = choose_patching_type.patching_type
            hook_name_func, hook_func = patching_types[patching_type]
            layer_labels, y_axis, patching_result_normalized_logits, patching_result_normalized_prs, patching_result_logits, patching_result_prs = run_patching(
                         patching_type=patching_type,
                         patching_hook_name_func=hook_name_func,
                         patching_hook_func=hook_func,
                         batch_size=choose_batch_size.batch_size,
                         fast_ssm=choose_fast_ssm.fast_ssm,
                         fast_conv=choose_fast_conv.fast_conv,
                         show_options=choose_show_options.show_options,
                         show_plot=False,
                         min_logit_diff=min_logit_diff,
                         max_logit_diff=max_logit_diff,
                         min_pr_diff=min_pr_diff,
                         max_pr_diff=max_pr_diff,
                         token_labels=token_labels,
                         prompt_uncorrupted_tokens=prompt_uncorrupted_tokens.view(1,-1),
                         uncorrupted_answer=uncorrupted_answer,
                         corrupted_answer=corrupted_answer)
            iters += 1
            if patching_result_normalized_logits_total is None:
                patching_result_normalized_logits_total = patching_result_normalized_logits
            else:
                patching_result_normalized_logits_total += patching_result_normalized_logits
        fig = imshow(patching_result_normalized_logits_total/iters, x=token_labels, y=layer_labels, xaxis="Position", yaxis=y_axis, title=f"Normalized logit difference after patching {patching_type} using hook {hook_name_func(layer='{layer}', position='{position}')}", font_size=8, show=False)
        plot_args_copy = dict(list(plot_args.items()))
        # this code can be used to make it taller so its more readable, if u plot them all
        if patching_type == CONV_FILTERS_PATCHING:
            plot_args_copy['height'] *= model.cfg.d_conv
        
        fig.update_layout(**plot_args_copy)
        fig.update_layout(legend=dict(
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=0.01
        ))
        fig.show()
        if patching_type == CONV_FILTERS_PATCHING: # also plot just layer 39
            layer_labels = [(i, label) for (i, label) in enumerate(layer_labels) if '39' in label]
            patching_result_normalized_logits_total = patching_result_normalized_logits_total[torch.tensor([i for (i, label) in layer_labels])]
            layer_labels = [label for (i,label) in layer_labels]
            plot_args_copy = dict(list(plot_args.items()))
            fig = imshow(patching_result_normalized_logits_total/iters, x=token_labels, y=layer_labels, xaxis="Position", yaxis=y_axis, title=f"Normalized logit difference after patching {patching_type} using hook {hook_name_func(layer='{layer}', position='{position}')}", font_size=8, show=False)
    
            fig.update_layout(**plot_args_copy)
            fig.update_layout(legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="left",
                x=0.01
            ))
            fig.show()
        

patching_button = ipywidgets.Button(description = 'Run Patching')
patching_button.on_click(do_patching)
display(patching_button)
    
# you can't just display stuff inside a widget callback, you need a wrap any display code in this
output = ipywidgets.Output()
display(output)

# Positional EAP with integrated gradients

In [ ]:
# REMEMBER TO MANUALLY PATCH TL LENS TO HAVE register_full_backward_hook

model_kwargs = {
    'fast_ssm': False,
    'fast_conv': False
}
BATCH_SIZE = 10

# removes all hooks including "leftover" ones that might stick around due to interrupting the model at certain times
clean_hooks(model)

def normalized_logit_diff_metric(patched_logits, unpatched_logits, corrupted_logits, patched_correct, corrupted_correct, also_return_acc=False):
    B,V = patched_logits.size()

    #print(data.unpatched.logits.size(), data.patched.logits.size(), data.corrupted.logits.size())
    A_logits_unpatched = unpatched_logits[torch.arange(B), patched_correct]
    A_logits_patched = patched_logits[torch.arange(B), patched_correct]
    A_logits_corrupted = corrupted_logits[torch.arange(B), patched_correct]

    B_logits_unpatched = unpatched_logits[torch.arange(B), corrupted_correct]
    B_logits_patched = patched_logits[torch.arange(B), corrupted_correct]
    B_logits_corrupted = corrupted_logits[torch.arange(B), corrupted_correct]

    # A and B are two potential outputs
    # if A patched > B patched, we are correct
    # else we are incorrect

    # thus we could just return A_logits_patched - B_logits_patched

    # however it is useful to "normalize" these values

    # in the worst case, our patching causes us to act like corrupted, and our diff will be
    # A_logits_corrupted - B_logits_corrupted
    # this will result in a small, negative value
    
    # in the best case, our patching will do nothing (cause us to act like unpatched), and our diff will be
    # A_logits_unpatched - B_logits_unpatched
    # this will result in a large, positive value

    # thus we can treat those as the "min" and "max" and normalize accordingly
    
    min_diff = A_logits_corrupted - B_logits_corrupted
    max_diff = A_logits_unpatched - B_logits_unpatched

    # the abs ensures that if it's wrong we don't try and make it more wrong
    possible_range = torch.abs(max_diff-min_diff)
    possible_range[possible_range == 0] = 1.0 # prevent divide by zero
    
    diff = A_logits_patched - B_logits_patched
    
    normalized_diff = (diff-min_diff)/possible_range

    # as described, 1.0 corresponds to acting like unpatched,
    # and 0.0 corresponds to acting like corrupted
    res = torch.mean(normalized_diff)
    
    if also_return_acc:
        num_correct = A_logits_patched > B_logits_patched
        acc = torch.sum(num_correct)/B
        return res, acc
    else:
        return res


# there's a subtle bug if you aren't careful:
# consider what happens when we do edge attribution patching and patch every edge
# what we want to happen is that it's identical to corrupted
# however this is not what happens!
# Start with layer 0:
# layer 0 will be patched
#    we subtract uncorrupted embed and add corrupted embed
#    in other words, the embed input to layer 0 will be from the corrupted run
# this results in layer 0 having an output of corrupted, as desired
# now consider layer 1
#    subtract uncorrupted embed and add corrupted embed
#      this is fine and results in embed input to layer 1 of corrupted
#    subtract uncorrupted layer 0 and add corrupted layer 0
#      layer 0 is already corrupted, so this has the effect of the output of layer 0 being
#          2*corrupted layer 0 - uncorrupted layer 0
#          this is not the same as corrupted!

# two ways to fix this:
# 1. fetch stored layer_input from uncorrupted run, and use that instead of the layer_input given in fwd_diff
#   this works, but then the gradients won't propogate properly (maybe? todo: test)
# 2. mark which edges are patched and don't "double patch" them
#   if we are patching all edges, this means that we simply apply only the embed diff to all layers,
#   as that'll result in all layers being patched
# 3. just subtract the outputs in the same forward pass, instead of a cached "unpatched" run
#  we do 3

global alpha
alpha = 1

def cache_output_hook( # hook_layer_output
    layer_output : Float[torch.Tensor, "B L D"],
    hook : HookPoint,
    layer : int,
    cached_outputs : Float[torch.Tensor, "NLayers+1 B L D"]):
    global alpha
    cached_outputs[layer+1] = layer_output
    return layer_output

def fwd_diff_hook( # hook_layer_input
    layer_input : Float[torch.Tensor, "B L D"],
    hook : HookPoint,
    layer : int,
    cached_outputs : Float[torch.Tensor, "NLayers+1 B L D"],
    corrupted_outputs : Float[torch.Tensor, "NLayers+1 B L D"]):
    global alpha
    return layer_input + (-cached_outputs[:layer+1]+corrupted_outputs[:layer+1]).sum(dim=0)*alpha

def bwd_diff_hook( # hook_layer_input
    grad : Float[torch.Tensor, "B L D"],
    hook: HookPoint,
    layer : int,
    batch_start: int,
    batch_end: int,
    cached_outputs : Float[torch.Tensor, "NLayers+1 B L D"],
    corrupted_outputs : Float[torch.Tensor, "NLayers+1 B L D"]):
    #print(f"running bwd with alpha {alpha} and layer {layer}") 
    
    # [N_upstream, B, L, D]
    upstream_diffs = (-cached_outputs[:layer+1]+corrupted_outputs[:layer+1])
    # grad is [B,L,D]

    # to do a taylor approximation of metric with respect to diff_0, we
    # multiply diffs and grad, then
    # sum over the L and D dimensions (this is doing a dot product of vectors of size L*D)
    # now we have attr of size [N_upstream, B, L]
    attr = (grad*upstream_diffs).sum(dim=-1)
    # [B, N_upstream, L]           [N_upstream, B, L]
    attr         =   torch.transpose(attr, 0, 1)
    # [B, N_upstream, L]
    attr = attr.clone().detach()
    if POSITIONS:
        attributions[batch_start:batch_end,:layer+1,layer+1] += attr
    else:
        #[B, N_upstream]
        attr = attr.sum(dim=-1)
        attributions[batch_start:batch_end,:layer+1,layer+1] += attr



def cache_conv_hook(
    conv_input: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    layer: int,
    cached_conv_inputs : Float[torch.Tensor, "NLayers B L E"]):
    cached_conv_inputs.append(conv_input)
    return conv_input

# for each conv slice,
# we have 
global filter_terms
def conv_patching_hook(
    conv_output: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    layer: int,
    cached_conv_inputs : Float[torch.Tensor, "NLayers B L E"],
    corrupted_conv_inputs : Float[torch.Tensor, "NLayers B L E"],
    slice_terms : [],
) -> Float[torch.Tensor, "B L E"]:
    ### This is identical to what the conv is doing
    # but we break it apart so we can patch on individual slices

    # we have two input hooks, the second one is the one we want
    D_CONV = model.cfg.d_conv

    # [E,1,D_CONV]
    conv_weight = model.blocks[layer].conv1d.weight
    # [E]
    conv_bias = model.blocks[layer].conv1d.bias
    uncorrupted_input = rearrange(cached_conv_inputs[layer], 'B L E -> B E L')
    corrupted_input = rearrange(corrupted_conv_inputs[layer], 'B L E -> B E L')
    B,E,L = uncorrupted_input.size()
    uncorrupted_input = torch.nn.functional.pad(uncorrupted_input, (D_CONV-1,0), mode='constant', value=0)
    corrupted_input = torch.nn.functional.pad(corrupted_input, (D_CONV-1,0), mode='constant', value=0)
    #print(uncorrupted_input.size(), corrupted_input.size())
    # we want some "patch hook" thing that uses autodiff or somethin
    output = torch.zeros([B,E,L], device=model.cfg.device)
    global alpha
    layer_slices = []
    for i in range(D_CONV):
        # [B,E,L]                      [E,1]                      [B,E,L]
        uncorrupted_contribution = conv_weight[:,0,i].view(E,1)*uncorrupted_input[:,:,i:i+L]
        corrupted_contribution = conv_weight[:,0,i].view(E,1)*corrupted_input[:,:,i:i+L]
        slice_term = uncorrupted_contribution*(1-alpha)+corrupted_contribution*(alpha)
        diff = -uncorrupted_contribution+corrupted_contribution
        slice_term.retain_grad()
        #slice_term.requires_grad = True
        layer_slices.append((diff, slice_term))
        output += slice_term
    output = rearrange(output, 'B E L -> B L E')
    output += conv_bias
    slice_terms.append(layer_slices)
    return output

def bwd_conv_hook(
    grad : Float[torch.Tensor, "B L E"], # we don't use this, hook is just so we are present in backward pass
    hook: HookPoint,
    layer : int,
    batch_start: int,
    batch_end: int,
    cached_conv_inputs : Float[torch.Tensor, "NLayers B L E"],
    corrupted_conv_inputs : Float[torch.Tensor, "NLayers B L E"],
    slice_terms: list # one list for each layer, each list contains one tensor for each conv slice
):
    # [B, L, E]
    #diffs = (-cached_conv_inputs[layer]+corrupted_conv_inputs[layer])
    layer_slice_terms = slice_terms[layer]
    for slice_i, (diff, slice_term) in enumerate(layer_slice_terms):
        # [B,E,L]
        slice_grad = slice_term.grad
        # dot product over the E dimension
        # [B,L]         [B,E,L]   [B,E,L]
        slice_attr = (slice_grad * diff).sum(dim=-2) # sum over E dimension
        if POSITIONS:
            # [B,L]
            conv_attributions[batch_start:batch_end,layer,slice_i] += slice_attr 
        else:
            #[B]
            slice_attr = slice_attr.sum(dim=-1)
            conv_attributions[batch_start:batch_end,layer,slice_i] += slice_attr

B,L = data.data.size()
# our data is pairs of unpatched, corrupted
n_patching_pairs = B//2
D_CONV = model.cfg.d_conv

# attributions[b,i+1,j+1] is the (i->j) edge attribution for patching pair b
# attributions[b,0,j] is the (embed->j) edge attribution for patching pair b
# attributions[b,i,-1] is the (i->output) edge attribution for patching pair b
POSITIONS = True
if POSITIONS:
    attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers+2, model.cfg.n_layers+2, L], device=model.cfg.device)
    conv_attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers, D_CONV, L], device=model.cfg.device)
else:
    attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers+2, model.cfg.n_layers+2], device=model.cfg.device)
    conv_attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers, D_CONV], device=model.cfg.device)



input_names = [f'blocks.{i}.hook_layer_input' for i in range(model.cfg.n_layers)]
output_names = [f'blocks.{i}.hook_out_proj' for i in range(model.cfg.n_layers)]
conv_input_names = [f'blocks.{i}.hook_in_proj' for i in range(model.cfg.n_layers)]
conv_names = [f'blocks.{i}.hook_conv' for i in range(model.cfg.n_layers)]

for batch_start in range(0, n_patching_pairs, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, n_patching_pairs)
    print(batch_start, batch_end)
    # we don't need grad for these forward passes
    torch.set_grad_enabled(False)
    embed_name = 'hook_embed'

    clean_hooks(model)
    # forward passes to get unpatched and corrupted
    unpatched_logits, unpatched_layer_outputs = model.run_with_cache(data.data[::2][batch_start:batch_end], names_filter=[embed_name] + output_names + conv_input_names, **model_kwargs)
    corrupted_logits, corrupted_layer_outputs = model.run_with_cache(data.data[1::2][batch_start:batch_end], names_filter=[embed_name] + output_names + conv_input_names, **model_kwargs)
    
    batch_size,L,D = unpatched_layer_outputs[output_names[0]].size()
    _,_,E = unpatched_layer_outputs[conv_input_names[0]].size()
    
    # get only the last token position (logit for next predicted token)
    # this is needed to support data of varying lengths
    unpatched_logits = unpatched_logits[torch.arange(batch_size), data.last_token_position[::2][batch_start:batch_end]]
    corrupted_logits = corrupted_logits[torch.arange(batch_size), data.last_token_position[1::2][batch_start:batch_end]]
    
    clean_hooks(model)

    # backward pass to compute grad of diff
    torch.set_grad_enabled(True)
        
    corrupted_outputs = torch.zeros([model.cfg.n_layers+1,batch_size,L,D], device=model.cfg.device)
    corrupted_conv_inputs = torch.zeros([model.cfg.n_layers,batch_size,L,E], device=model.cfg.device)
    corrupted_outputs.requires_grad = False
    # first one is for embed
    corrupted_outputs[0] = corrupted_layer_outputs[embed_name]
    for layer in range(model.cfg.n_layers):
        output_name = output_names[layer]
        corrupted_outputs[layer+1] = corrupted_layer_outputs[output_name]
        corrupted_conv_inputs[layer] = corrupted_layer_outputs[conv_input_names[layer]]    
    # cleanup
    del corrupted_layer_outputs
    
    for param in model.parameters():
        param.requires_grad = True
        param.grad = None # reset grads
    
    last_layer = model.cfg.n_layers-1
    # forward pass to do partial patches
    cached_outputs = torch.zeros([model.cfg.n_layers+1,batch_size,L,D], device=model.cfg.device)
    cached_outputs.requires_grad = False
    
    # EAP layer hooks
    cache_output_hooks = [(embed_name,
                           partial(cache_output_hook,
                                   layer=-1,
                                   cached_outputs=cached_outputs))]
    
    cache_output_hooks += [(output_names[layer],
                            partial(cache_output_hook,
                                    layer=layer,
                                    cached_outputs=cached_outputs)) for layer in range(model.cfg.n_layers)]
    
    fwd_hooks = cache_output_hooks
    fwd_hooks += [(input_names[layer],
                  partial(
                      fwd_diff_hook,
                      layer=layer,
                      cached_outputs=cached_outputs,
                      corrupted_outputs=corrupted_outputs,
                  )) for layer in range(model.cfg.n_layers)]
    bwd_hooks = [(input_names[layer],
                  partial(bwd_diff_hook,
                          layer=layer,
                          cached_outputs=cached_outputs,
                          corrupted_outputs=corrupted_outputs,
                          batch_start=batch_start,
                          batch_end=batch_end)) for layer in range(model.cfg.n_layers)]
    # extra hook for the very last layer
    fwd_hooks.append((f'blocks.{last_layer}.hook_resid_post',
                      partial(fwd_diff_hook,
                              layer=last_layer+1,
                              cached_outputs=cached_outputs,
                              corrupted_outputs=corrupted_outputs,
                             )))
    bwd_hooks.append((f'blocks.{last_layer}.hook_resid_post',
                      partial(bwd_diff_hook,
                              layer=last_layer+1,
                              cached_outputs=cached_outputs,
                              corrupted_outputs=corrupted_outputs,
                              batch_start=batch_start,
                              batch_end=batch_end,
                             )))

    # Conv hooks
    cached_conv_inputs = []
    slice_terms = []

    fwd_hooks += [(conv_input_names[layer],
                  partial(
                      cache_conv_hook,
                      layer=layer,
                      cached_conv_inputs=cached_conv_inputs,
                  )) for layer in range(model.cfg.n_layers)]
    
    
    fwd_hooks += [(conv_names[layer],
                  partial(
                      conv_patching_hook,
                      layer=layer,
                      cached_conv_inputs=cached_conv_inputs,
                      corrupted_conv_inputs=corrupted_conv_inputs,
                      slice_terms=slice_terms,
                  )) for layer in range(model.cfg.n_layers)]
    bwd_hooks += [(input_names[layer], # anywhere before the slice terms works, so we'll just pick start of layer since that backward is called
                  partial(
                      bwd_conv_hook,
                      layer=layer,
                      cached_conv_inputs=cached_conv_inputs,
                      corrupted_conv_inputs=corrupted_conv_inputs,
                      slice_terms=slice_terms,
                      batch_start=batch_start,
                      batch_end=batch_end,
                  )) for layer in range(model.cfg.n_layers)]
    
    for fwd in fwd_hooks:
        model.add_hook(*fwd, "fwd")

    for bwd in bwd_hooks:
        model.add_hook(*bwd, "bwd")
    
    # with integrated gradients
    # simply sums over doing "partial patches" like 0.2 patch and 0.8 unpatched 
    # ITERS = 1 is just edge attribution patching (without integraded gradients)
    ITERS = 5
    for i in range(ITERS+1):
        global alpha
        # alpha ranges from 0 to 1
        if ITERS > 1:
            alpha = i/float(ITERS-1)
        elif ITERS == 1: # no integrated gradients, set alpha to 1
            alpha = 1.0

        # it tries to propogate gradients to these, detach them
        slice_terms.clear()
        cached_conv_inputs.clear()
        torch.cuda.empty_cache()
        cached_outputs[:] = 0
        cached_outputs.grad = None
        cached_outputs.detach_()
        corrupted_outputs.grad = None
        corrupted_outputs.detach_()
        #cached_conv_inputs.grad = None
        #cached_conv_inputs.detach_()
        corrupted_conv_inputs.grad = None
        corrupted_conv_inputs.detach_()
        model.zero_grad()
        for param in model.parameters():
            param.grad = None
        if i == ITERS:
            torch.cuda.empty_cache()
            break # we just use this for cleaning up
        logits = model(data.data[::2][batch_start:batch_end], **model_kwargs)
        logits = logits[torch.arange(batch_size), data.last_token_position[::2][batch_start:batch_end]]
        metric = normalized_logit_diff_metric(
            patched_logits=logits,
            unpatched_logits=unpatched_logits,
            corrupted_logits=corrupted_logits,
            patched_correct=data.correct[::2][batch_start:batch_end][:,0],
            corrupted_correct=data.correct[1::2][batch_start:batch_end][:,0]
        )
        print(f"alpha {alpha} metric {metric}")
        # run backward pass, which adds to attributions
        metric.backward()
        #conv_attrs = conv_attributions.mean(dim=0)
        #print(conv_attrs)
        #attrs = attributions.mean(dim=0)
        #print(attrs)

# todo: maybe the diffs should have alpha in the backward pass? No, that would mean alpha of 0 gives all zero attrs

# average over all the samples
attributions[:] = attributions[:]/ITERS
conv_attributions[:] = conv_attributions[:]/ITERS


# don't need grad for rest of this
torch.set_grad_enabled(False)

clean_hooks(model)

# Binary Search to find Circuit

In [ ]:
model_kwargs = {
    'fast_ssm': False,
    'fast_conv': False
}

global storage
def storage_hook(
    x,
    hook: HookPoint,
    **kwargs,
):
    global storage
    #if hook.name == 'hook_embed':
    #    for k in list(storage.keys()):
    #        del storage[k]
    storage[hook.name] = x
    return x

def resid_patching_hook(
    x,
    hook: HookPoint,
    input_hook_name: str,
    batch_start: int,
    batch_end: int,
    position: int = None,
):
    global storage
    x_uncorrupted = storage[input_hook_name][batch_start:batch_end:2]
    x_corrupted = storage[input_hook_name][batch_start+1:batch_end:2]
    if position is None: # if position not specified, apply to all positions
        x[batch_start:batch_end:2] = x[batch_start:batch_end:2] - x_uncorrupted + x_corrupted
    else:
        x[batch_start:batch_end:2,position] = x[batch_start:batch_end:2,position] - x_uncorrupted[:,position] + x_corrupted[:,position]
    return x

def overwrite_patching_hook(
    x,
    hook: HookPoint,
    input_hook_name: str,
    batch_start: int,
    batch_end: int,
    position: int = None,
):
    x_corrupted = x[batch_start+1:batch_end:2]
    if position is None: # if position not specified, apply to all positions
        x[batch_start:batch_end:2] = x_corrupted
    else:
        if x_corrupted.size()[1] != L: raise ValueError(f'warning: in hook {hook.name} with input_hook_name {input_hook_name} you are patching on position in the second index but size is {x_corrupted.size()}')
        x[batch_start:batch_end:2,position] = x_corrupted[:,position]
    return x


def overwrite_h_hook(
    x,
    hook: HookPoint,
    input_hook_name: str,
    batch_start: int,
    batch_end: int,
    position: int = None,
):
    x[batch_start:batch_end:2] = x[batch_start+1:batch_end:2]
    return x

################## CONV ##############

# we do a hacky thing where this first hook clears the global storage
# second hook stores all the hooks
# then third hook computes the output (over all the hooks)
# this avoids recomputing and so is much faster
CONV_HOOKS = "conv hooks"
CONV_BATCHES = "conv batches"
def conv_patching_init_hook(
    x,
    hook: HookPoint,
    batch_start: int,
    batch_end: int,
    **kwargs
):
    # we need to clear this here
    # i tried having a "current layer" variable in the conv_storage that only clears when it doesn't match
    # but that doesn't work if you only patch the same layer over and over,
    # as stuff gets carried over
    # this way of doing things is much safer and lets us assume it'll be empty
    # well not quite, note that conv_patching_hook will be called with different batch_start and batch_end inputs during one forward pass
    # so we need to account for that in the keys we use
    global conv_storage
    conv_storage = {CONV_BATCHES: set()}
    return x

# hook h has a weird index!!!!!

def conv_patching_storage_hook(
    x,
    hook: HookPoint,
    conv_filter_i: int,
    position: int,
    layer: int,
    batch_start: int,
    batch_end: int,
    **kwargs,
):
    global storage
    storage[hook.name] = x
    global conv_storage
    hooks_key = (CONV_HOOKS, batch_start, batch_end)
    if not hooks_key in conv_storage:
        conv_storage[hooks_key] = [] # we can't do this above because it'll be emptied again on the next batch before this is called
    conv_storage[hooks_key].append({"position": position, "conv_filter_i": conv_filter_i})
    conv_storage[CONV_BATCHES].add((batch_start, batch_end))
    return x


global storage_for_grad

global conv_storage
def conv_patching_hook(
    conv_output: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    input_hook_name: str,
    layer: int,
    **kwargs,
) -> Float[torch.Tensor, "B L E"]:
    global conv_storage
    global storage
    ### This is identical to what the conv is doing
    # but we break it apart so we can patch on individual filters

    # we have two input hooks, the second one is the one we want
    input_hook_name = input_hook_name[1]
    
    D_CONV = model.cfg.d_conv

    # given batches like [(2,4), (5,6)] and total size 7 this returns (0,2), (4,5), (6,7) 
    def get_missing_batches(batches, B):
        covered_i = torch.zeros([B])
        for batch_start, batch_end in batches:
            covered_i[batch_start:batch_end] = 1
    
        missing_batches = []
        missing_start = 0
        for i in range(B):
            if covered_i[i] == 1:
                if i != missing_start:
                    missing_batches.append((missing_start, i))
                missing_start = i+1
        if covered_i[B-1] == 0:
            missing_batches.append((missing_start, B))
        return missing_batches
    

    
    # [E,1,D_CONV]
    conv_weight = model.blocks[layer].conv1d.weight
    # [E]
    conv_bias = model.blocks[layer].conv1d.bias
    
    # don't recompute these if we don't need to
    # because we stored all the hooks and batches in conv_storage, we can just do them all at once
    output_key = f'output' # they need to share an output because they write to the same output tensor
    if not output_key in conv_storage:
        #print("layer", layer, "keys", conv_storage)
        apply_to_all_hooks = [] # this is important because otherwise the [0:None] would overwrite the previous results (or vice versa)
        apply_to_all_key = (CONV_HOOKS, 0, None)
        if apply_to_all_key in conv_storage:
            apply_to_all_hooks = conv_storage[apply_to_all_key]
            # we need to do this so it applies to the other batches that we aren't otherwise patching
            for batch_start, batch_end in get_missing_batches(conv_storage[CONV_BATCHES], conv_output.size()[0]):
                conv_storage[CONV_BATCHES].add(batch_start, batch_end)
                conv_storage[(CONV_HOOKS, batch_start, batch_end)] = []
        for batch_start, batch_end in conv_storage[CONV_BATCHES]:
            if batch_start == 0 and batch_end == None: continue # we cover this in the apply to all hooks above
            def get_filter_key(i):
                return f'filter_{i}'
            conv_input_uncorrupted = storage[input_hook_name][batch_start:batch_end:2]
            conv_input_corrupted = storage[input_hook_name][batch_start+1:batch_end:2]
            B, L, E = conv_input_uncorrupted.size()
            
            conv_input_uncorrupted = rearrange(conv_input_uncorrupted, 'B L E -> B E L')
            conv_input_corrupted = rearrange(conv_input_corrupted, 'B L E -> B E L')
            
            # pad zeros in front
            # [B,E,D_CONV-1+L]
            padded_input_uncorrupted = torch.nn.functional.pad(conv_input_uncorrupted, (D_CONV-1,0), mode='constant', value=0)
            padded_input_corrupted = torch.nn.functional.pad(conv_input_corrupted, (D_CONV-1,0), mode='constant', value=0)
    
            # compute the initial filter values
            for i in range(D_CONV):
                filter_key = get_filter_key(i)
                # [B,E,L]                      [E,1]                      [B,E,L]
                filter_contribution = conv_weight[:,0,i].view(E,1)*padded_input_uncorrupted[:,:,i:i+L]
                conv_storage[filter_key] = filter_contribution
            
            # apply all the hooks
            for hook in conv_storage[(CONV_HOOKS, batch_start, batch_end)] + apply_to_all_hooks:
                position = hook['position']
                conv_filter_i = hook['conv_filter_i']
                #print(f"position {position} conv_filter_i {conv_filter_i} batch_start {batch_start} batch_end {batch_end}")
                filter_key = get_filter_key(conv_filter_i)
                # [1,E,L]                                   [E,1]                          # [B,E,L]
                corrupted_filter_contribution = conv_weight[:,0,conv_filter_i].view(E,1)*padded_input_corrupted[:,:,conv_filter_i:conv_filter_i+L]
                filter_contribution = conv_storage[filter_key]
                if position is None:
                    # [B,E,L]                    [B,E,L]
                    filter_contribution = corrupted_filter_contribution
                else:
                    # [B,E]                                                  [B,E]
                    filter_contribution[:,:,position] = corrupted_filter_contribution[:,:,position]
                conv_storage[filter_key] = filter_contribution
            
            # compute the output
            output = sum([conv_storage[get_filter_key(i)] for i in range(D_CONV)])
            #output = torch.zeros([B,E,L], device=model.cfg.device)
            #print(f'B {B} B2 {B2} E {E} L {L} conv_storage keys {conv_storage.keys()} filter sizes {[(k,v.size()) for (k,v) in conv_storage.items() if not type(v) is int]}')
            for i in range(D_CONV):
                filter_key = get_filter_key(i)
                #output += conv_storage[filter_key]
                del conv_storage[filter_key] # clean up now we are done with it, just to be safe
                
            # bias is not dependent on input so no reason to patch on it, just apply it as normal
            output += conv_bias.view(E, 1)
            output = rearrange(output, 'B E L -> B L E')
            # interleave it back with the corrupted as every other
            conv_output[batch_start:batch_end:2] = output
        conv_storage[output_key] = conv_output
    return conv_storage[output_key]




limited_layers = list(range(model.cfg.n_layers))

## Setup edges for ACDC
edges = []
B,L = data.data.size()
#positions = list(range(L)) # 

if POSITIONS:
    positions = list(range(L))
else:
    positions = [None]

INPUT_HOOK = f'hook_embed'
INPUT_NODE = 'embed'

last_layer = model.cfg.n_layers-1
OUTPUT_HOOK = f'blocks.{last_layer}.hook_resid_post'
OUTPUT_NODE = 'output'

def input(layer):
    return f'{layer}.i'

def output(layer):
    return f'{layer}.o'

def conv(layer):
    return f'{layer}.conv'

def skip(layer):
    return f'{layer}.skip'

def ssm(layer):
    return f'{layer}.ssm'

# important to have storage be global and not passed into the hooks! Otherwise it gets very slow (tbh, i don't know why)
global storage
storage = {}


attrs = attributions.mean(dim=0)
conv_attrs = conv_attributions.mean(dim=0)
ALWAYS_KEEP_WEIGHT = -10
for pos in positions:
    # direct connections from embed to output
    edges.append(Edge(
            label=str(pos),
            input_node=INPUT_NODE,
            input_hook=(INPUT_HOOK, storage_hook),
            output_node=OUTPUT_NODE,
            output_hook=(OUTPUT_HOOK, partial(resid_patching_hook, position=pos)),
            score_diff_when_patched=attrs[0,model.cfg.n_layers+1,pos].flatten()[0],
    ))

for layer in limited_layers:
    for pos_i, pos in enumerate(positions):
        # edge from embed to layer input
        edges.append(Edge(
                label=str(pos),
                input_node=INPUT_NODE,
                input_hook=(INPUT_HOOK, partial(storage_hook)),
                output_node=input(layer),
                output_hook=(f'blocks.{layer}.hook_layer_input', partial(resid_patching_hook, position=pos)),
                score_diff_when_patched=attrs[0,layer+1,pos].flatten()[0],
        ))


        # input to conv
        for conv_i in range(model.cfg.d_conv):
            edges.append(Edge(
                    label=(f'[{pos}:{conv_i-model.cfg.d_conv+1}]'.replace("None:", "")), # [-D_CONV+1, -D_CONV+2, ..., -2, -1, 0]
                    input_node=input(layer),
                    input_hook=[
                        (f'blocks.{layer}.hook_layer_input', conv_patching_init_hook),
                        (f'blocks.{layer}.hook_in_proj', partial(conv_patching_storage_hook, position=pos, layer=layer, conv_filter_i=conv_i))
                    ],
                    output_node=conv(layer),
                    output_hook=(f'blocks.{layer}.hook_conv', partial(conv_patching_hook, position=pos, layer=layer, conv_filter_i=conv_i)),
                    score_diff_when_patched=conv_attrs[layer,conv_i,pos].flatten()[0],
            ))
        
        # conv to ssm
        if pos is None:
            # we need a seperate hook for each pos, but put them all into one edge
            hooks = []
            for other_pos in range(L):
                hooks.append((f'blocks.{layer}.hook_h.{other_pos}', overwrite_h_hook))
            edges.append(Edge(
                    input_node=conv(layer),
                    output_node=ssm(layer),
                    output_hook=hooks,
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT,
            ))
        else:
            edges.append(Edge(
                    label=f'{pos}',
                    input_node=conv(layer),
                    output_node=ssm(layer),
                    output_hook=(f'blocks.{layer}.hook_h.{pos}', overwrite_h_hook),
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT
            ))

        if pos_i == 0: # we only need one of these
            # ssm to output
            edges.append(Edge(
                    input_node=ssm(layer),
                    output_node=output(layer),
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT
            ))
        
        # input to skip
        edges.append(Edge(
                label=f'{pos}',
                input_node=input(layer),
                output_node=skip(layer),
                output_hook=(f'blocks.{layer}.hook_skip', partial(overwrite_patching_hook, position=pos)),
                score_diff_when_patched=ALWAYS_KEEP_WEIGHT
        ))

        if pos_i == 0: # we only need one of these
            # skip to output
            edges.append(Edge(
                    input_node=skip(layer),
                    output_node=output(layer),
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT
            ))
        
        for later_layer in limited_layers:
                if layer < later_layer:
                    # edge from layer output to other layer input
                    edges.append(Edge(
                            label=str(pos),
                            input_node=output(layer),
                            input_hook=(f'blocks.{layer}.hook_out_proj', storage_hook),
                            output_node=input(later_layer),
                            output_hook=(f'blocks.{later_layer}.hook_layer_input', partial(resid_patching_hook, position=pos)),
                            score_diff_when_patched=attrs[layer+1,later_layer+1,pos].flatten()[0],
                    ))
        
        # edge from layer output to final layer output
        edges.append(Edge(
                label=str(pos),
                input_node=output(layer),
                input_hook=(f'blocks.{layer}.hook_out_proj', storage_hook),
                output_node=OUTPUT_NODE,
                output_hook=(OUTPUT_HOOK, partial(resid_patching_hook, position=pos)),
                score_diff_when_patched=attrs[layer+1,model.cfg.n_layers+1,pos].flatten()[0],
        ))



def normalized_logit_diff_acdc_metric(data: ACDCEvalData, printing=False):
    B,V = data.patched.logits.size()

    # [batch_size]
    patched_correct = data.patched.correct[:,0]
    #print(data.unpatched.logits.size(), data.patched.logits.size(), data.corrupted.logits.size())
    A_logits_unpatched = data.unpatched.logits[torch.arange(B), patched_correct]
    A_logits_patched = data.patched.logits[torch.arange(B), patched_correct]
    A_logits_corrupted = data.corrupted.logits[torch.arange(B), patched_correct]

    corrupted_correct = data.corrupted.correct[:,0]
    B_logits_unpatched = data.unpatched.logits[torch.arange(B), corrupted_correct]
    B_logits_patched = data.patched.logits[torch.arange(B), corrupted_correct]
    B_logits_corrupted = data.corrupted.logits[torch.arange(B), corrupted_correct]

    # A and B are two potential outputs
    # if A patched > B patched, we are correct
    # else we are incorrect

    # thus we could just return A_logits_patched - B_logits_patched

    # however it is useful to "normalize" these values

    # in the worst case, our patching causes us to act like corrupted, and our diff will be
    # A_logits_corrupted - B_logits_corrupted
    # this will result in a small, negative value
    
    # in the best case, our patching will do nothing (cause us to act like unpatched), and our diff will be
    # A_logits_unpatched - B_logits_unpatched
    # this will result in a large, positive value

    # thus we can treat those as the "min" and "max" and normalize accordingly
    
    min_diff = A_logits_corrupted - B_logits_corrupted
    max_diff = A_logits_unpatched - B_logits_unpatched

    possible_range = (max_diff-min_diff)
    possible_range[possible_range == 0] = 1.0 # prevent divide by zero
    
    diff = A_logits_patched - B_logits_patched
    normalized_diff = (diff-min_diff)/torch.abs(possible_range) # abs prevents incorrect data from wanting to be more incorrect

    if printing:
        print(f"A corrupted {A_logits_corrupted}")
        print(f"B corrupted {B_logits_corrupted}")
        print(f"A unpatched {A_logits_unpatched}")
        print(f"B unpatched {B_logits_unpatched}")
        print(f"A patched {A_logits_patched}")
        print(f"B patched {B_logits_patched}")
        print(f"min diff {min_diff}")
        print(f"max diff {max_diff}")
        print(f"possible range {possible_range}")
        print(f"diff {diff}")
        print(f"normalized diff {normalized_diff}")
    # as described, 1.0 corresponds to acting like unpatched,
    # and 0.0 corresponds to acting like corrupted

    return torch.mean(normalized_diff)
import acdc
cfg = ACDCConfig(
    ckpt_directory = "blah",
    thresh = 0.0001,
    rollback_thresh = 0.0001,
    metric=normalized_logit_diff_acdc_metric,
    # extra inference args
    model_kwargs=model_kwargs,
    # these are needed for doing graph pruning
    input_node=INPUT_NODE,
    output_node=OUTPUT_NODE,
    # batch size for evaluating data points
    batch_size=1,
    log_level=LOG_LEVEL_INFO,
    # if False, will be equivalent to batch_size=1
    batched = True,
    # set these two to false to use traditional ACDC
    # recursive will try patching multiple at a time (this is faster sometimes)
    recursive = True,
    # try_patching_multiple_at_same_time will evaluate many different patchings before commiting to any
    # and includes a rollback scheme if after patching one, the others get worse
    try_patching_multiple_at_same_time = True,
    ## if true, you metric will also have the logits from a run with no patching available
    # (useful for normalized logit diff)
    store_unpatched_logits = True,
)
from acdc import get_currently_patched_edge_hooks, eval_acdc, wrap_run_with_hooks, get_logits_of_predicted_next_token
unpatched_logits = get_logits_of_predicted_next_token(
        model=model,
        data=data.valid_data,
        last_token_position=data.valid_last_token_position,
        **model_kwargs
)

valid_unpatched_logits = get_logits_of_predicted_next_token(
        model=model,
        data=data.valid_data,
        last_token_position=data.valid_last_token_position,
        **model_kwargs
)

def eval_edges(edges_keeping, all_edges, valid=False):
    for edge in all_edges:
        edge.patching = True
        edge.checked = True
    for edge in edges_keeping:
        edge.patching = False
        edge.checked = True
    currently_patched_edge_hooks = get_currently_patched_edge_hooks(cfg=cfg, edges=edges)
    if valid:
        return eval_acdc(
                model=wrap_run_with_hooks(model=model, fwd_hooks=currently_patched_edge_hooks, **cfg.model_kwargs),
                data=data.valid_data,
                last_token_position=data.valid_last_token_position,
                correct=data.valid_correct,
                incorrect=data.valid_incorrect,
                metric=cfg.metric,
                num_edges=1,
                constrain_to_answers=data.constrain_to_answers,
                unpatched_logits=valid_unpatched_logits)[0].item()
    else:
        return eval_acdc(
            model=wrap_run_with_hooks(model=model, fwd_hooks=currently_patched_edge_hooks, **cfg.model_kwargs),
            data=data.data,
            last_token_position=data.last_token_position,
            correct=data.correct,
            incorrect=data.incorrect,
            metric=cfg.metric,
            num_edges=1,
            constrain_to_answers=data.constrain_to_answers,
            unpatched_logits=unpatched_logits)[0].item()

# can also do this instead to include negative contributions but I find the graph is larger, ymmv
#edges.sort(key=lambda x: -abs(x[0]))
edges.sort(key=lambda edge: edge.score_diff_when_patched)

import math
def test_pos(pos):
    print(f"testing pos {pos}")
    edges_to_keep = edges[:pos]
    metric = eval_edges(edges_to_keep, edges)
    print(f"testing pos {pos} got metric {metric}")
    return metric

# from https://en.wikipedia.org/wiki/Binary_search_algorithm
def binary_search(n, T):
    L = 0
    R = n - 1
    while L != R:
        m = math.ceil((L + R) / 2)
        if test_pos(m) > T:
            R = m - 1
        else:
            L = m
    # go one further because this gives us below thresh
    return min(n-1, L+1)

ACC_THRESH = 0.4
torch.set_grad_enabled(False)
'''
cutoff = binary_search(len(edges), T=ACC_THRESH)

edges_to_keep = edges[:cutoff]
scores = [edge.score_diff_when_patched for edge in edges[:cutoff]]
print(f"keeping top {cutoff} edges")
metric = eval_edges(edges_to_keep, edges)
print(f"got metric {metric}")    

metric = eval_edges(edges_to_keep, edges, valid=True)
print(f"got valid metric {metric}")    
# .85 has 3061 edges for 5 ITERS
# .85 has 2876 edges for 30 ITERS


# patching format 0:
# 2162 edges (2016 always keep)
'''
num_always_keep = 0
for edge in edges:
    if edge.score_diff_when_patched == ALWAYS_KEEP_WEIGHT:
        num_always_keep += 1
print(f"always keep {num_always_keep}")

In [ ]:
for i in range(2,10):
    target = i/10.0
    cutoff = binary_search(len(edges), T=target)
    edges_to_keep = edges[:cutoff]
    adj_mat = compute_adj_mat(edges_to_keep)
    pruned_adj_mat = better_prune_edges(adj_mat)

    position_map = {}
    for l in range(L):
        position_map[l] = f'pos{l}{toks[l]}'

    import graphviz
    for layer in range(model.cfg.n_layers):
        layer_attrs = []
        for l in range(L):
            for filter_i in range(model.cfg.d_conv):
                filter_offset = -filter_i
                index = l*model.cfg.d_conv + filter_i
                attr = by_filters[index, layer]
                if attr < 0:
                    layer_attrs.append((position_map[l+filter_offset], attr, filter_offset))
        for tok, attr, filter_offset in layer_attrs:
            attr = -attr*1000

    def map_layer(layer):
        if layer == 0: return 'embed'
        if layer == model.cfg.n_layers+1: return 'output'
        else: return str(layer-1)

    for layer in range(model.cfg.n_layers+2):
        layer_attrs = []
        for other_layer in range(model.cfg.n_layers+2):
            for pos in range(L):
                if pruned_adj_mat[pos, layer, other_layer] != 0:
                    layer_attrs.append((position_map[pos], f"->{map_layer(other_layer)}", pruned_adj_mat[pos, layer, other_layer]))
                if pruned_adj_mat[pos, other_layer, layer] != 0:
                    layer_attrs.append((position_map[pos], f"{map_layer(other_layer)}->", pruned_adj_mat[pos, other_layer, layer]))
        for tok, label, attr in layer_attrs:
            attr = -attr*1000


    def layer_to_i(layer):
        if layer == 'embed': return 0
        elif layer == 'output': return model.cfg.n_layers+1
        else: return int(layer)+1

    forbidden = [0,1]
    pruned_adj_mat = better_prune_edges(adj_mat)
    for i in forbidden:
        pruned_adj_mat[:,i,:] = 0
        pruned_adj_mat[:,:,i] = 0
    present_layers = set()
    for i in range(model.cfg.n_layers):
        for j in range(model.cfg.n_layers):
            if not i+1 in forbidden and not j+1 in forbidden:
                if torch.any(pruned_adj_mat[:,i+1,j+1] != 0):
                    present_layers.add(i)
                    present_layers.add(j)
                    #print(f"present edge {i}->{j}")
    pruned_dot = graphviz.Digraph('graph')
    for edge in edges_to_keep:
        is_between_layers, layer_input, layer_output = between_layers_info(edge)
        if is_between_layers:
            ini, outi = layer_to_i(layer_input), layer_to_i(layer_output)
            pos = int(edge.label)
            if pruned_adj_mat[pos, ini, outi] != 0:
                if not ini in forbidden and not outi in forbidden:
                    pruned_dot.edge(edge.input_node, edge.output_node, label=position_map[int(edge.label)])
        else:
            layer = edge.input_node.split(".")[0]
            if not layer in ['embed', 'output']:
                layer = int(layer)
            if layer in present_layers: # don't display stuff that are disconnected
                if '.ssm' in edge.output_node or '.skip' in edge.output_node or '.skip' in edge.input_node: # don't need all these since they are all forced on
                    if edge.label == 0 or edge.label == "" or edge.label is None or edge.label == "0":
                        pruned_dot.edge(edge.input_node, edge.output_node)
                else:
                    pruned_dot.edge(edge.input_node, edge.output_node, label=edge.label)
    from pathlib import Path
    Path(f"seed {seed} no embed or zero/").mkdir(parents=True, exist_ok=True)
    output_name = f"seed {seed} no embed or zero/pruned dot {target}"
    # too big:
    from IPython.display import Image, FileLink
    pruned_dot.render(output_name, format="png") # it automatically appends png
    #display(Image(filename=output_name + ".png"))
    #display(FileLink(output_name + ".png"))

# Prune and Display EAP Results

In [ ]:
edges = edges_to_keep
L = data.data.size()[1]
print(f"got L of {L}")
if POSITIONS:
    by_filters = torch.zeros([model.cfg.d_conv*L, model.cfg.n_layers])
else:
    by_filters = torch.zeros([model.cfg.d_conv, model.cfg.n_layers])

# change this to get the different plots
CAP = 0.0001
# 1.0 is boolean (present or not)
# 0.0 is just give score

def filter_score(score):
    if CAP == 0.0: return score
    if CAP == 1.0: return 1.0
    return max(score, -CAP)
def torch_filter_score(score):
    if CAP == 0.0: return score
    if CAP == 1.0: return (score != 0) + 0.0
    return torch.clamp(score, min=-CAP)
for edge in edges:
    if not edge.patching and edge.checked:
        if '.conv' in edge.output_node:
            if ":" in edge.label:
                pos = int(edge.label.split(":")[0][1:])
                rest = edge.label.split(":")[1]
            else:
                pos = None
                rest = edge.label[1:]
            filter = int(rest.split("]")[0])        
            layer = edge.output_node.split(".")[0]
            if POSITIONS:
                try:
                    by_filters[model.cfg.d_conv*pos + abs(int(filter)),int(layer)] = filter_score(edge.score_diff_when_patched)
                except:
                    print(L, pos, abs(int(filter)))
                    raise
            else:
                by_filters[abs(int(filter)),int(layer)] = filter_score(edge.score_diff_when_patched)
        if '.skip' in edge.output_node and False:        
            layer = edge.output_node.split(".")[0]
            by_filters[model.cfg.d_conv,int(layer)] = filter_score(edge.score_diff_when_patched)
        if '.ssm' in edge.output_node and False:
            layer = edge.output_node.split(".")[0]
            by_filters[model.cfg.d_conv+1,int(layer)] = filter_score(edge.score_diff_when_patched)
            

def layer_to_i(node):
    if node == INPUT_NODE:
        return 0
    elif node == OUTPUT_NODE:
        return model.cfg.n_layers+1 # because embed is 0
    else:
        return int(node)+1 # because embed is 0


def between_layers_info(edge):
    if edge.patching: return False, None, None
    is_between_layers = False
    layer_input = None
    layer_output = None
    if edge.input_node == INPUT_NODE:
        is_between_layers = True
        layer_input = INPUT_NODE
    if edge.output_node == OUTPUT_NODE:
        is_between_layers = True
        layer_output = OUTPUT_NODE

    if '.' in edge.input_node:
        input_layer, input_type = edge.input_node.split(".")
        if edge.input_node == output(input_layer):
            is_between_layers = True
            layer_input = str(input_layer)
    if '.' in edge.output_node:
        output_layer, output_type = edge.output_node.split(".")
        if edge.output_node == input(output_layer):
            is_between_layers = True
            layer_output = str(output_layer)
    return is_between_layers, layer_input, layer_output

def compute_adj_mat(edges):
    if POSITIONS:
        adj_mat = torch.zeros([L, model.cfg.n_layers+2, model.cfg.n_layers+2])
    else:
        adj_mat = torch.zeros([model.cfg.n_layers+2, model.cfg.n_layers+2])
    for edge in edges:
        is_between_layers, layer_input, layer_output = between_layers_info(edge)
        if is_between_layers:
            if POSITIONS:
                pos = int(edge.label)
                adj_mat[pos, layer_to_i(layer_input), layer_to_i(layer_output)] = edge.score_diff_when_patched
            else:
                adj_mat[layer_to_i(layer_input), layer_to_i(layer_output)] = edge.score_diff_when_patched
    return adj_mat


L = data.data.size()[1]
import networkx as nx
def better_get_nx_graph(adj_mat) -> nx.DiGraph:
    '''
    Converts the edges into a networkx graph
    only edges that have checked == True and patching == False are included
    if include_unchecked=True, any edge that has checked == False is also included
    '''
    num_nodes = adj_mat.size()[-1]
    G = nx.DiGraph()
    edges = []
    if POSITIONS:
        for pos in range(L):
            for i in range(num_nodes):
                for j in range(num_nodes):
                    if adj_mat[pos,i,j] != 0:
                        G.add_edge(str(i), str(j))
                        edges.append((i,j,pos,adj_mat[pos,i,j]))
    else:
        for i in range(num_nodes):
            for j in range(num_nodes):
                if adj_mat[i,j] != 0:
                    G.add_edge(str(i), str(j))
                    edges.append((i,j,None,adj_mat[i,j]))
    return G, edges

def better_prune_edges(adj_mat):
    num_nodes = adj_mat.size()[-1]
    input_node = 0
    output_node = num_nodes-1
    import networkx as nx
    G, edges = better_get_nx_graph(adj_mat=adj_mat)
    pruned_adj_mat = torch.zeros(adj_mat.size())
    pruned_edges = []
    for i,j,pos,attr in edges:
        connected_to_input = False
        try:
            to_input = nx.shortest_path(G, source=str(input_node), target=str(i))
            connected_to_input = True
        except nx.NetworkXNoPath:
            pass
        except nx.NodeNotFound:
            raise ValueError(f"Graph does not have input node {input_node}")
        
        connected_to_output = False
        try:
            to_output = nx.shortest_path(G, source=str(j), target=str(output_node))
            connected_to_output = True
        except nx.NetworkXNoPath:
            pass
        except nx.NodeNotFound:
            raise ValueError(f"Graph does not have output node {output_node}")
        if connected_to_input and connected_to_output:
            pruned_adj_mat[pos,i,j] = attr
        else:
            print(f"pruning {pos} {i}->{j} {connected_to_input} {connected_to_output}")
    return pruned_adj_mat

adj_mat = compute_adj_mat(edges_to_keep)
pruned_adj_mat = better_prune_edges(adj_mat)

def to_str(lis):
    return [str(x) for x in lis]
x_labels = to_str([-x for x in range(model.cfg.d_conv)])
toks = model.to_str_tokens(data.data[0])
x_labels_all_pos = []
for l in range(L):
    x_labels_all_pos += [label + toks[l+int(label)] + "_" + str(l) for label in x_labels]
print(x_labels, by_filters.size())
print(len(x_labels_all_pos))
title = f'which parts of layer using (conv filters, skip, and/or conv), capped to {-CAP}'
if CAP == 1.0:
    title = f'which parts of layer using (conv filters, skip, and/or conv)'
print(x_labels_all_pos)
imshow(by_filters.T, y=to_str(range(model.cfg.n_layers)), x=x_labels_all_pos, title=title, font_size=7, fix_size=True)
#for layer in range(model.cfg.n_layers):
#    imshow(by_filters[:,layer:layer+1].T, y=to_str([layer]), x=x_labels_all_pos, title=str(layer), font_size=9)

labels = ['embed'] + [str(x) for x in range(model.cfg.n_layers)] + ['output']
if POSITIONS:
    for position in range(L):
        if torch.any(torch.abs(pruned_adj_mat[position]) > CAP/10.0):
            imshow(torch_filter_score(pruned_adj_mat[position]), y=labels, x=labels, title=f'pos {position} tok {toks[position]} adjacency matrix clamped to {CAP}', font_size=13, zmin=-CAP, zmax=CAP, fix_size=True, height=800)
else:
    imshow(pruned_adj_mat, y=labels, x=labels, title=f'adjacency matrix clamped to {CAP}', font_size=8)


In [ ]:
position_map = {}
for l in range(L):
    position_map[l] = f'pos{l}{toks[l]}'

import graphviz
for layer in range(model.cfg.n_layers):
    layer_attrs = []
    for l in range(L):
        for filter_i in range(model.cfg.d_conv):
            filter_offset = -filter_i
            index = l*model.cfg.d_conv + filter_i
            attr = by_filters[index, layer]
            if attr < 0:
                layer_attrs.append((position_map[l+filter_offset], attr, filter_offset))
    print(layer)
    for tok, attr, filter_offset in layer_attrs:
        attr = -attr*1000
        print(f"  {tok}[{filter_offset}] {attr:.5f}")

def map_layer(layer):
    if layer == 0: return 'embed'
    if layer == model.cfg.n_layers+1: return 'output'
    else: return str(layer-1)

for layer in range(model.cfg.n_layers+2):
    layer_attrs = []
    for other_layer in range(model.cfg.n_layers+2):
        for pos in range(L):
            if pruned_adj_mat[pos, layer, other_layer] != 0:
                layer_attrs.append((position_map[pos], f"->{map_layer(other_layer)}", pruned_adj_mat[pos, layer, other_layer]))
            if pruned_adj_mat[pos, other_layer, layer] != 0:
                layer_attrs.append((position_map[pos], f"{map_layer(other_layer)}->", pruned_adj_mat[pos, other_layer, layer]))
    print(map_layer(layer))
    for tok, label, attr in layer_attrs:
        attr = -attr*1000
        print(f"  {tok} {label} {attr:.5f}")


def layer_to_i(layer):
    if layer == 'embed': return 0
    elif layer == 'output': return model.cfg.n_layers+1
    else: return int(layer)+1

forbidden = [0,1,40]
pruned_adj_mat = better_prune_edges(adj_mat)
for i in forbidden:
    pruned_adj_mat[:,i,:] = 0
    pruned_adj_mat[:,:,i] = 0
present_layers = set()
for i in range(model.cfg.n_layers):
    for j in range(model.cfg.n_layers):
        if not i+1 in forbidden and not j+1 in forbidden:
            if torch.any(pruned_adj_mat[:,i+1,j+1] != 0):
                present_layers.add(i)
                present_layers.add(j)
                #print(f"present edge {i}->{j}")
print(present_layers)
pruned_dot = graphviz.Digraph('graph')
for edge in edges_to_keep:
    is_between_layers, layer_input, layer_output = between_layers_info(edge)
    if is_between_layers:
        ini, outi = layer_to_i(layer_input), layer_to_i(layer_output)
        pos = int(edge.label)
        if pruned_adj_mat[pos, ini, outi] != 0:
            if not ini in forbidden and not outi in forbidden:
                pruned_dot.edge(edge.input_node, edge.output_node, label=position_map[int(edge.label)])
    else:
        layer = int(edge.input_node.split(".")[0])
        if layer in present_layers: # don't display stuff that are disconnected
            if '.ssm' in edge.output_node or '.skip' in edge.output_node or '.skip' in edge.input_node: # don't need all these since they are all forced on
                if edge.label == 0 or edge.label == "" or edge.label is None or edge.label == "0":
                    pruned_dot.edge(edge.input_node, edge.output_node)
            else:
                pruned_dot.edge(edge.input_node, edge.output_node, label=edge.label)
output_name = f'pruned dot {ACC_THRESH}'
# too big:
from IPython.display import Image, FileLink
pruned_dot.render(output_name, format="png") # it automatically appends png
display(Image(filename=output_name + ".png"))
display(FileLink(output_name + ".png"))

# 

# SAE EAP

## Load SAES

In [4]:

# load SAEs
import sys
if not "/home/dev/sae-k-sparse-mamba" in sys.path:
    sys.path.append("/home/dev/sae-k-sparse-mamba")
import os
os.chdir('/home/dev/sae-k-sparse-mamba')
saes = [None] # don't do layer 0 because that clutters the attributions
from sae.sae import Sae

SAE_LAYERS = list(range(1,42))

ckpt_dir = "/home/dev/sae-k-sparse-mamba/"
for i in SAE_LAYERS:
    print(i)
    hook = f'blocks.{i}.hook_out_proj'
    path = [ckpt_dir + f for f in sorted(list(os.listdir(ckpt_dir))) if hook in f][0] + "/" + f'hook_{hook}.pt'
    #path = f'/home/dev/sae-k-sparse-mamba/blocks.{i}.hook_resid_pre/hook_blocks.{i}.hook_resid_pre.pt'
    print(path)
    if i != 34:
        saes.append(None)
    else:
        saes.append(Sae.load_from_disk(path, hook=f'blocks.{i}.hook_out_proj', device=model.cfg.device))

    


1
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry190.txtblocks.1.hook_out_proj/hook_blocks.1.hook_out_proj.pt
2
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry191.txtblocks.2.hook_out_proj/hook_blocks.2.hook_out_proj.pt
3
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry192.txtblocks.3.hook_out_proj/hook_blocks.3.hook_out_proj.pt
4
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry193.txtblocks.4.hook_out_proj/hook_blocks.4.hook_out_proj.pt
5
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry194.txtblocks.5.hook_out_proj/hook_blocks.5.hook_out_proj.pt
6
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry195.txtblocks.6.hook_out_proj/hook_blocks.6.hook_out_proj.pt
7
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry196.txtblocks.7.hook_out_proj/hook_blocks.7.hook_out_proj.pt
8
/home/dev/sae-k-sparse-mamba/0.0001414213562373095 initialTry200.txtblocks.8.hook_out_proj/hook_blocks.8.hook_out_proj.pt
9
/home/

## Eval with SAEs

In [ ]:
def sae_hook(
    x,
    hook,
    layer,
):
    B,L,D = x.size()
    K = D//2
    sae = saes[layer]
    uncorrupted_features = sae.encode(x)
    weighted_features = uncorrupted_features
    top_acts, top_indices = weighted_features.topk(K, sorted=False)
    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    return sae_out


sae_hooks = [(f'blocks.{layer}.hook_out_proj', partial(sae_hook, layer=layer)) for layer in SAE_LAYERS]

model_kwargs = {
    "fast_ssm": True,
    "fast_conv": True
}

from acdc import accuracy_metric, wrap_run_with_hooks
acc = data.eval(model=wrap_run_with_hooks(model=model, fwd_hooks=sae_hooks, **model_kwargs),
                batch_size=10,
                metric=accuracy_metric).mean()
acc_no_sae = data.eval(model=wrap_run_with_hooks(model=model, fwd_hooks=[], **model_kwargs),
                batch_size=10,
                metric=accuracy_metric).mean()
print(f"patching with SAEs, got accuracy {acc}")
print(f"baseline accuracy without patches, {acc_no_sae}")

from mamba_lens.input_dependent_hooks import clean_hooks

for i in SAE_LAYERS:
    clean_hooks(model)
    sae_hooks = [(f'blocks.{i}.hook_out_proj', partial(sae_hook, layer=i)) for layer in range(1)]
    #sae_hooks = [(f'hook_embed', partial(sae_hook, layer=layer)) for layer in range(i,i+1)]
    
    model_kwargs = {
        "fast_ssm": True,
        "fast_conv": True
    }
    
    from acdc import accuracy_metric, wrap_run_with_hooks
    acc = data.eval(model=wrap_run_with_hooks(model=model, fwd_hooks=sae_hooks, **model_kwargs),
                    batch_size=10,
                    metric=accuracy_metric).mean()
    print(f"layer {i}")
    print(f"patching with SAEs, got accuracy {acc}")
    #break




# SAE EAP

In [5]:
# from tqdm import tqdm
from functools import partial
from jaxtyping import Float
from transformer_lens.hook_points import HookPoint
from mamba_lens.input_dependent_hooks import clean_hooks
from tqdm import tqdm
from einops import rearrange

SAE_LAYERS = [34]

model_kwargs = {
    'fast_ssm': False,
    'fast_conv': True
}
BATCH_SIZE = 20

# removes all hooks including "leftover" ones that might stick around due to interrupting the model at certain times
clean_hooks(model)

def normalized_logit_diff_metric(patched_logits, unpatched_logits, corrupted_logits, patched_correct, corrupted_correct, also_return_acc=False):
    B,V = patched_logits.size()

    #print(data.unpatched.logits.size(), data.patched.logits.size(), data.corrupted.logits.size())
    A_logits_unpatched = unpatched_logits[torch.arange(B), patched_correct]
    A_logits_patched = patched_logits[torch.arange(B), patched_correct]
    A_logits_corrupted = corrupted_logits[torch.arange(B), patched_correct]

    B_logits_unpatched = unpatched_logits[torch.arange(B), corrupted_correct]
    B_logits_patched = patched_logits[torch.arange(B), corrupted_correct]
    B_logits_corrupted = corrupted_logits[torch.arange(B), corrupted_correct]

    # A and B are two potential outputs
    # if A patched > B patched, we are correct
    # else we are incorrect

    # thus we could just return A_logits_patched - B_logits_patched

    # however it is useful to "normalize" these values

    # in the worst case, our patching causes us to act like corrupted, and our diff will be
    # A_logits_corrupted - B_logits_corrupted
    # this will result in a small, negative value
    
    # in the best case, our patching will do nothing (cause us to act like unpatched), and our diff will be
    # A_logits_unpatched - B_logits_unpatched
    # this will result in a large, positive value

    # thus we can treat those as the "min" and "max" and normalize accordingly
    
    min_diff = A_logits_corrupted - B_logits_corrupted
    max_diff = A_logits_unpatched - B_logits_unpatched

    # the abs ensures that if it's wrong we don't try and make it more wrong
    possible_range = torch.abs(max_diff-min_diff) # todo: try removing this
    possible_range[possible_range == 0] = 1.0 # prevent divide by zero
    
    diff = A_logits_patched - B_logits_patched
    normalized_diff = (diff-min_diff)/possible_range
    
    # as described, 1.0 corresponds to acting like unpatched,
    # and 0.0 corresponds to acting like corrupted
    res = torch.mean(normalized_diff)
    
    if also_return_acc:
        num_correct = A_logits_patched > B_logits_patched
        acc = torch.sum(num_correct)/B
        return res, acc
    else:
        return res

global alpha
alpha = 1

global cached_outputs
global corrupted_outputs


####################### SAE HOOKS ##################

global sae_terms
global corrupted_sae_inputs


# TODO: maybe try flowing gradient to all values instead of just topK? idk
def sae_hook(
    x,
    hook,
    layer,
    saving
):
    global sae_terms
    global alpha
    global corrupted_sae_inputs
    # s is [B,L,E]
    K = saes[layer].cfg.k
    #K = 32
    sae = saes[layer]
    token_outs = []
    layer_sae_terms = []
    B,L,D = x.size()
    if saving=='blend':
        uncorrupted_features = sae.encode(x)
        corrupted_features = sae.encode(corrupted_sae_inputs[sae_layer_to_i[layer]])
        weighted_features = uncorrupted_features*(1-alpha) + corrupted_features * alpha
        weighted_features.retain_grad()
    elif saving=='basic':
        uncorrupted_features = sae.encode(x)
        weighted_features = uncorrupted_features
    elif saving=='cursed':
        corrupted_features = sae.encode(corrupted_sae_inputs[sae_layer_to_i[layer]])
        weighted_features = corrupted_features
    top_acts, top_indices = weighted_features.topk(K, sorted=False)

    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    if saving == 'blend':
        diff = (-uncorrupted_features + corrupted_features)
        #print(f"diff layer {layer} {torch.linalg.norm(diff[:,0], ord=2)}")
        sae_terms.append((diff, weighted_features, top_indices))
    return sae_out
    

def bwd_sae_hook(
    grad,
    hook,
    layer,
    batch_start,
    batch_end):
    #print(f'{layer}')
    global sae_terms
    # [B, L, E]
    #diffs = (-cached_conv_inputs[layer]+corrupted_conv_inputs[layer])
    (diff, weighted_features, top_indices) = sae_terms[sae_layer_to_i[layer]] # Because we ignore 0
    # weighted_features is size [B,L,NFeatures]
    # diff is size [B,L,NFeatures]
    #attr = weighted_features.grad*diff
    B,L,D = diff.size()
    #i1, i2, i3 = get_batched_index_into(top_indices)
    global result_sae_attrs
    global result_sae_counts
    #attrs = (*).view(top_indices.size())
    # [NLayers,L,D]
    EPSILON = 0.00001
    for b in range(B):
        # [L,K]
        with torch.no_grad():
            indices = top_indices[b]
            iL, iK = get_index_into(indices)
            attrs = diff[b,iL,iK]*weighted_features.grad[b,iL,iK]
            attrs[torch.abs(attrs)<EPSILON] = 0 # numerical imprecision makes things that should be zero slightly not zero
            # [L,NFeatures]
            result_sae_attrs[sae_layer_to_i[layer],iL,iK] += attrs
            result_sae_counts[sae_layer_to_i[layer],iL,iK] += (torch.abs(attrs)>=EPSILON) + 0 # only increment those that are bigger than eps

########## Compute Attributions ###############
B,L = data.data.size()
# our data is pairs of unpatched, corrupted
n_patching_pairs = B//2
D_CONV = model.cfg.d_conv

POSITIONS = True
NFeatures = saes[SAE_LAYERS[0]].W_dec.size()[0]

sae_layer_to_i = dict([(layer, i) for (i,layer) in enumerate(SAE_LAYERS)])

global result_sae_attrs
global result_sae_counts
result_sae_attrs = torch.zeros([len(SAE_LAYERS), L, NFeatures], device=model.cfg.device)
result_sae_counts = torch.zeros([len(SAE_LAYERS), L, NFeatures], device=model.cfg.device)
if POSITIONS:
    attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers+2, model.cfg.n_layers+2, L], device=model.cfg.device)
    conv_attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers, D_CONV, L], device=model.cfg.device)
    # just a few defaultdicts
    sae_attributions = defaultdict(lambda: # layer 
                                   defaultdict(lambda: # batch
                                                       defaultdict(lambda: # token pos
                                                                   defaultdict(lambda: # feature index
                                                                               0)))) # attr
    sae_counts = defaultdict(lambda: # layer 
                                   defaultdict(lambda: # batch
                                                       defaultdict(lambda: # token pos
                                                                   defaultdict(lambda: # feature index
                                                                               0)))) # count
else:
    attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers+2, model.cfg.n_layers+2], device=model.cfg.device)
    conv_attributions = torch.zeros([n_patching_pairs, model.cfg.n_layers, D_CONV], device=model.cfg.device)
    sae_attributions = defaultdict(lambda: # layer 
                                   defaultdict(lambda: # batch
                                                       defaultdict(lambda: # feature index
                                                                   defaultdict(lambda: 0)))) # attr
    sae_counts = defaultdict(lambda: # layer 
                                   defaultdict(lambda: # batch
                                                       defaultdict(lambda: # feature index
                                                                   defaultdict(lambda: 0)))) # count

global sae_attr
sae_attr = []
input_names = [f'blocks.{i}.hook_layer_input' for i in range(model.cfg.n_layers)]
sae_input_names = [f'blocks.{i}.hook_out_proj' for i in range(model.cfg.n_layers)]
output_names = [f'blocks.{i}.hook_out_proj' for i in range(model.cfg.n_layers)]

def caching_sae_hook(x,
                 hook,
                layer):
    
    global corrupted_sae_inputs
    corrupted_sae_inputs[sae_layer_to_i[layer]] = x.clone()
    return x

sae_fwd_hooks = [(sae_input_names[layer],
                  partial(
                      sae_hook,
                      layer=layer,
                      saving='basic',
                  )) for layer in SAE_LAYERS]
sae_fwd_hooks_corrupted = [(sae_input_names[layer],
                  partial(
                      sae_hook,
                      layer=layer,
                      saving='cursed',
                  )) for layer in SAE_LAYERS]

caching_hooks = [(sae_input_names[layer], partial(caching_sae_hook, layer=layer)) for layer in SAE_LAYERS]
# THIS ORDER IS IMPORTANT
sae_corruption_hooks = caching_hooks + sae_fwd_hooks

# todo: maybe duplicate confuses it?
for batch_start in range(0, n_patching_pairs, BATCH_SIZE):
    batch_end = min(batch_start + BATCH_SIZE, n_patching_pairs)
    print(batch_start, batch_end)
    # we don't need grad for these forward passes
    torch.set_grad_enabled(False)
    embed_name = 'hook_embed'
    clean_hooks(model)
    # forward passes to get unpatched and corrupted
    #unpatched_logits = model(data.data[::2][batch_start:batch_end], **model_kwargs)
    unpatched_logits = model.run_with_hooks(data.data[::2][batch_start:batch_end], fwd_hooks=sae_fwd_hooks, **model_kwargs)
    batch_size,L,_ = unpatched_logits.size()
    names_to_store = sorted(list(set([embed_name] + output_names + sae_input_names))) # no duplicates
    # for our other types of patching, this is identical to applying all the patches
    # however for sae's that is not the case (as they are lossy)
    # make sure to apply them in that case
    global corrupted_sae_inputs    
    D = model.cfg.d_model
    corrupted_sae_inputs = torch.zeros([len(SAE_LAYERS), batch_size,L,D], device=model.cfg.device)
    # this strategy results in non-zero diff for the first token, which is not what we want since there is no difference 
    # cache them, then insert those in so this is identical to alpha 1.0
    #corrupted_logits = model.run_with_hooks(data.data[1::2][batch_start:batch_end], fwd_hooks=caching_hooks, **model_kwargs)
    # pass in original as that's what happens when alpha 1.0
    #corrupted_logits = model.run_with_hooks(data.data[::2][batch_start:batch_end], fwd_hooks=sae_fwd_hooks_corrupted, **model_kwargs)
    # corrupted should just be cached when apply sae
    # VERY IMPORTANT THAT CACHING HOOKS COME BEFORE SAE FWD HOOKS (otherwise will cache the result after SAE which is not what we want as that'll double apply later)
    #corrupted_logits = model.run_with_hooks(data.data[1::2][batch_start:batch_end], fwd_hooks=caching_hooks+sae_fwd_hooks, **model_kwargs)
    #corrupted_logits = model.run_with_hooks(data.data[::2][batch_start:batch_end], fwd_hooks=sae_fwd_hooks_corrupted, **model_kwargs)
    corrupted_logits = model.run_with_hooks(data.data[1::2][batch_start:batch_end], fwd_hooks=caching_hooks, **model_kwargs)
    
    batch_size,L,_ = corrupted_logits.size()
    
    #batch_size,L,D = corrupted_layer_outputs[output_names[0]].size()
    #_,_,E = corrupted_layer_outputs[conv_input_names[0]].size()
    
    # get only the last token position (logit for next predicted token)
    # this is needed to support data of varying lengths
    unpatched_logits = unpatched_logits[torch.arange(batch_size), data.last_token_position[::2][batch_start:batch_end]]
    corrupted_logits = corrupted_logits[torch.arange(batch_size), data.last_token_position[1::2][batch_start:batch_end]]
    
    clean_hooks(model)

    torch.set_grad_enabled(True)
    
    for param in model.parameters():
        param.requires_grad = True
        param.grad = None # reset grads
    
    last_layer = model.cfg.n_layers-1
    # forward pass to do partial patches
    cached_outputs = torch.zeros([model.cfg.n_layers+1,batch_size,L,D], device=model.cfg.device)
    cached_outputs.requires_grad = False
    
    ######### EAP layer hooks
    # sae hooks
    global sae_terms
    sae_terms = []
    fwd_hooks = [(sae_input_names[layer],
                  partial(
                      sae_hook,
                      layer=layer,
                      saving='blend',
                  )) for layer in SAE_LAYERS]
    
    bwd_hooks = [(f'blocks.{layer}.hook_resid_pre', # anywhere before the slice terms works, so we'll just pick start of layer since that backward is called
                  partial(
                      bwd_sae_hook,
                      layer=layer,
                      batch_start=batch_start,
                      batch_end=batch_end,
                  )) for layer in SAE_LAYERS]


    # add the hooks
    
    clean_hooks(model)
    for fwd in fwd_hooks:
        model.add_hook(*fwd, "fwd")

    for bwd in bwd_hooks:
        model.add_hook(*bwd, "bwd")
    
    # with integrated gradients
    # simply sums over doing "partial patches" like 0.2 patch and 0.8 unpatched 
    # ITERS = 1 is just edge attribution patching (without integraded gradients)
    ITERS = 5
    for i in range(ITERS+1):
        global alpha
        # alpha ranges from 0 to 1
        if ITERS > 1:
            alpha = i/float(ITERS-1)
        elif ITERS == 1: # no integrated gradients, set alpha to 1
            alpha = 1.0

        # it tries to propogate gradients to these, detach them
        sae_terms.clear()
        torch.cuda.empty_cache()
        cached_outputs[:] = 0
        cached_outputs.grad = None
        cached_outputs.detach_()
        corrupted_sae_inputs.grad = None
        corrupted_sae_inputs.detach_()
        model.zero_grad()
        for param in model.parameters():
            param.grad = None
        if i == ITERS:
            torch.cuda.empty_cache()
            break # we just use this for cleaning up
        logits = model(data.data[::2][batch_start:batch_end], **model_kwargs)
        logits = logits[torch.arange(batch_size), data.last_token_position[::2][batch_start:batch_end]]
        metric,acc = normalized_logit_diff_metric(
            patched_logits=logits,
            unpatched_logits=unpatched_logits,
            corrupted_logits=corrupted_logits,
            patched_correct=data.correct[::2][batch_start:batch_end][:,0],
            corrupted_correct=data.correct[1::2][batch_start:batch_end][:,0],
            also_return_acc=True
        )
        print(f"alpha {alpha} metric {metric} acc {acc}")
        # run backward pass, which adds to attributions
        metric.backward()
        #conv_attrs = conv_attributions.mean(dim=0)
        #print(conv_attrs)
        #attrs = attributions.mean(dim=0)
        #print(attrs)
    # average over all the iters
    
    #sae_attr[-1] = torch.mean(torch.stack(sae_attr[-1]), dim=0)

# todo: maybe the diffs should have alpha in the backward pass? No, that would mean alpha of 0 gives all zero attrs

# average over all the samples


# don't need grad for rest of this
torch.set_grad_enabled(False)

clean_hooks(model)

# of note: maybe the diff*grad doesn't make sense, they have different argmaxes, we should really be working in original feature space or somethin

# we don't get 0.0 for alpha of 1.0 because "fully applying the patch" doesn't apply it to embed or zero, so it's only going to look like corrupted for
# so really, my corrupted should be only a partial patch
# the layers that we patch on
# I did this it's very cursed but I did it and now it's zero for alpha 1.0 as (desired??)

# don't divide by zero
result_sae_counts[result_sae_counts==0] = 1.0
# average
result_sae_attrs /= result_sae_counts


# MAYBE I SHOULD TRY SCALING EVERYTHING? SO ROUNDING ISSUES CAUSE LESS PROBLEM?

0 20


/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py:1344: UserWarning: Using a non-full backward hook when the forward contains multiple autograd Nodes is deprecated and will be removed in future versions. This hook will be missing some grad_input. Please use register_full_backward_hook to get the documented behavior.
  warnings.warn("Using a non-full backward hook when the forward contains multiple autograd Nodes "


alpha 0.0 metric 1.0 acc 1.0
alpha 0.25 metric 0.9782790541648865 acc 1.0
alpha 0.5 metric 0.9524111151695251 acc 1.0
alpha 0.75 metric 0.924477756023407 acc 1.0
alpha 1.0 metric 0.8985557556152344 acc 1.0
20 40
alpha 0.0 metric 1.0 acc 1.0
alpha 0.25 metric 0.976324737071991 acc 1.0
alpha 0.5 metric 0.9469570517539978 acc 1.0
alpha 0.75 metric 0.9145744442939758 acc 1.0
alpha 1.0 metric 0.8856685757637024 acc 1.0
40 60
alpha 0.0 metric 1.0 acc 1.0
alpha 0.25 metric 0.9737293124198914 acc 1.0
alpha 0.5 metric 0.9377811551094055 acc 1.0
alpha 0.75 metric 0.8978981375694275 acc 1.0
alpha 1.0 metric 0.8646696209907532 acc 1.0
60 80
alpha 0.0 metric 1.0 acc 1.0
alpha 0.25 metric 0.9764624834060669 acc 1.0
alpha 0.5 metric 0.9442872405052185 acc 1.0
alpha 0.75 metric 0.9128745198249817 acc 1.0
alpha 1.0 metric 0.8820773363113403 acc 1.0
80 100
alpha 0.0 metric 1.0 acc 1.0
alpha 0.25 metric 0.9747900366783142 acc 1.0
alpha 0.5 metric 0.94378262758255 acc 1.0
alpha 0.75 metric 0.9111061096191

In [73]:
torch.save((result_sae_counts, result_sae_attrs), "feature_attrs.pkl")

In [6]:
#del result_sae_counts, result_sae_attrs
#result_sae_counts, result_sae_attrs = torch.load("feature_attrs.pkl")

model_kwargs = {
    'fast_ssm': True,
    'fast_conv': True
}

total = data.data.size()[0]//2*ITERS
#otal = 4160*20
#total = 20000*25
original_total = 400*5
# 200 works well
original_prop = 200/float(original_total)
new_prop = total*original_prop
from mamba_lens.input_dependent_hooks import clean_hooks
clean_hooks(model)
import sys
import torch
import os
import sys
import wandb
import os
import signal
import acdc
from tqdm import tqdm
from typing import Any  
from transformer_lens.hook_points import HookPoint


from functools import partial


###################################### SAE ###########################


# we do a hacky thing where this first hook clears the global storage
# second hook stores all the hooks
# then third hook computes the output (over all the hooks)
# this avoids recomputing and so is much faster
SAE_HOOKS = "sae hooks"
SAE_BATCHES = "sae batches"
SAE_OUTPUT = "sae output"
def sae_patching_storage_hook(
    x,
    hook: HookPoint,
    sae_feature_i: int,
    dummy: bool,
    position: int,
    layer: int,
    batch_start: int,
    batch_end: int,
    **kwargs,
):
    global sae_storage
    if not SAE_HOOKS in sae_storage:
        sae_storage[SAE_HOOKS] = [] # we can't do this above because it'll be emptied again on the next batch before this is called
    sae_storage[SAE_OUTPUT] = None # clear output
    sae_storage[SAE_HOOKS].append({"position": position, "sae_feature_i": sae_feature_i, "dummy": dummy})
    #print(f"sae feature i {sae_feature_i} position {position} layer {layer}")
    return x

from jaxtyping import Float
from einops import rearrange

global sae_storage
sae_storage = {}
def sae_patching_hook(
    x: Float[torch.Tensor, "B L E"],
    hook: HookPoint,
    input_hook_name: str,
    layer: int,
    **kwargs,
) -> Float[torch.Tensor, "B L E"]:
    global sae_storage
    ### This is identical to what the conv is doing
    # but we break it apart so we can patch on individual filters
    # we have two input hooks, the second one is the one we want
    input_hook_name = input_hook_name[1]
    # don't recompute these if we don't need to
    # because we stored all the hooks and batches in conv_storage, we can just do them all at once
    
    # they need to share an output because they write to the same output tensor
    if sae_storage[SAE_OUTPUT] is None:
        #print(f"running for layer {layer}")
        K = saes[layer].cfg.k
        sae = saes[layer]
        #print(f"layer {layer} storage {sae_storage}")
        sae_output = torch.zeros(x.size(), device=model.cfg.device)
        #print("layer", layer, "keys", conv_storage)
        def get_filter_key(i):
            return f'filter_{i}'
        sae_input_uncorrupted = x[::2]
        sae_input_corrupted = x[1::2]
        B, L, D = sae_input_uncorrupted.size()
        for l in range(L):
            # [B, NFeatures]                             [B,D]
            uncorrupted_features = sae.encode(sae_input_uncorrupted[:,l])
            # [B, NFeatures]                             [B,D]
            corrupted_features = sae.encode(sae_input_corrupted[:,l])
            patched_features = corrupted_features.clone()
            #patched_features = torch.zeros(corrupted_features.size(), device=model.cfg.device) # patch everything except the features we are keeping around
            # apply hooks (one hook applies to a single feature)
            #print(f"{len(sae_storage[SAE_HOOKS])} hooks")
            
            for hook_data in sae_storage[SAE_HOOKS]:
                position = hook_data['position']
                sae_feature_i = hook_data['sae_feature_i']
                dummy = hook_data['dummy']
                if not dummy and (position == l or position is None): # position is None means all positions
                    if copy_from_other:
                        patched_features[:,sae_feature_i] = corrupted_features[:,sae_feature_i]
                    else:
                        patched_features[:,sae_feature_i] = uncorrupted_features[:,sae_feature_i]
                    
                    #print(f"applying sae feature {sae_feature_i} to position {position} for layer {layer}")
                    #uncorrupted_features[:,sae_feature_i] = corrupted_features[:,sae_feature_i]
            
            # compute sae outputs
            patched_top_acts, patched_top_indices = patched_features.topk(K, sorted=False)
            corrupted_top_acts, corrupted_top_indices = corrupted_features.topk(K, sorted=False)      
            sae_output[::2,l] = sae.decode(patched_top_acts, patched_top_indices)     
            sae_output[1::2,l] = sae.decode(corrupted_top_acts, corrupted_top_indices)
        sae_storage = {} # clean up and prepare for next layer
        sae_storage[SAE_OUTPUT] = sae_output # store the output
    return sae_storage[SAE_OUTPUT]



limited_layers = list(range(model.cfg.n_layers))


from acdc.data.ioi import ioi_data_generator, ABC_TEMPLATES, get_all_single_name_abc_patching_formats
from acdc.data.utils import generate_dataset

num_patching_pairs = 200
seed = 27
valid_seed = 28
constrain_to_answers = True
# this makes our data size 800, first 400 is each (a,b) pair, and then second 400 is each pair swapped to be (b,a)
has_symmetric_patching = True


## Setup edges for ACDC
edges = []
B,L = data.data.size()
#positions = list(range(L)) # 

if POSITIONS:
    positions = list(range(L))
else:
    positions = [None]

INPUT_HOOK = f'hook_embed'
INPUT_NODE = 'embed'

last_layer = model.cfg.n_layers-1
OUTPUT_HOOK = f'blocks.{last_layer}.hook_resid_post'
OUTPUT_NODE = 'output'

def input(layer):
    return f'{layer}.i'

def output(layer):
    return f'{layer}.o'

def sae(layer):
    return f'{layer}.sae'

def conv(layer):
    return f'{layer}.conv'

def skip(layer):
    return f'{layer}.skip'

def ssm(layer):
    return f'{layer}.ssm'

# important to have storage be global and not passed into the hooks! Otherwise it gets very slow (tbh, i don't know why)
global storage
storage = {}



#attrs = attributions.mean(dim=0)
#conv_attrs = conv_attributions.mean(dim=0)
ALWAYS_KEEP_WEIGHT = -10

ONLY_SAE = True
def remove_non_sae(w):
    return ALWAYS_KEEP_WEIGHT

print("creating edges:")
for layer in limited_layers:
    print(layer)
    #print(f'layer {layer}')
    for pos_i, pos in enumerate(positions):
        num_copy = 0
        #print(f'position {pos}')
        # edge from embed to layer input
        if layer in SAE_LAYERS:
            # this is a bit gross in that we rely on the order that layers are inputted (so the edges above this have hooks applied before us, and later have hooks applied after us)
            #if pos is None:
            #    available_features = all_used_layer_sae_features[layer]
            #else:
            #    available_features = all_used_layer_sae_features[layer][pos]
            #available_features = sorted(list(available_features)) # deterministic
            #print(f'{len(available_features)} features')
            #new_prop = 10
            #available_features = torch.where(torch.logical_and(result_sae_counts[sae_layer_to_i[layer],pos]>new_prop,result_sae_attrs[sae_layer_to_i[layer],pos] != 0.0))[0]
            available_features = torch.where(torch.logical_and(result_sae_counts[sae_layer_to_i[layer],pos]>12,result_sae_attrs[sae_layer_to_i[layer],pos] != 0.0))[0]
            print(f"layer {layer} pos {pos} features {len(available_features)}")
            for sae_feature_i in available_features:
                # multiply attr by count (idk why but I find this gives less edges)
                attr = result_sae_attrs[sae_layer_to_i[layer],pos,sae_feature_i].flatten()[0].item()*result_sae_counts[sae_layer_to_i[layer],pos,sae_feature_i].flatten().item()
                #attr = result_sae_attrs[sae_layer_to_i[layer],pos,sae_feature_i].flatten()[0].item()
                copy_from_other = False
                #if attr == 0.0:
                #    copy_from_other = True
                #    attr = ALWAYS_KEEP_WEIGHT
                #    num_copy += 1
                edges.append(Edge(
                        label=(f'[{pos}:{sae_feature_i}]'.replace("None:", "")),
                        input_node=input(layer),
                        input_hook=[
                            (f'blocks.{layer}.hook_resid_pre', partial(sae_patching_storage_hook, position=pos, dummy=False, layer=layer, copy_from_other=copy_from_other, sae_feature_i=sae_feature_i))
                        ],
                        output_node=sae(layer),
                        output_hook=(f'blocks.{layer}.hook_out_proj', partial(sae_patching_hook, position=pos, layer=layer)),
                        score_diff_when_patched=attr,
                ))
                edges[-1].original_attr = result_sae_attrs[sae_layer_to_i[layer],pos,sae_feature_i].flatten()[0].item()
                edges[-1].original_count = result_sae_counts[sae_layer_to_i[layer],pos,sae_feature_i].flatten().item()
            if num_copy > 0:
                print(f"layer {layer} pos {pos} num copy {num_copy}")
            for i in range(100):
                if not i in available_features:
                    keep_i = i
                    break
            # we need one that always exists to ensure that saes are always applied
            edges.append(Edge(
                    label=(f'[{pos}:KEEP]'.replace("None:", "")),
                    input_node=input(layer),
                    input_hook=[
                        (f'blocks.{layer}.hook_resid_pre', partial(sae_patching_storage_hook, position=pos, layer=layer, dummy=True, sae_feature_i=-1))
                    ],
                    output_node=sae(layer),
                    output_hook=(f'blocks.{layer}.hook_out_proj', partial(sae_patching_hook, position=pos, layer=layer)),
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT,
            ))
        else:
            # skip to output
            edges.append(Edge(
                    input_node=input(layer),
                    output_node=sae(layer),
                    score_diff_when_patched=ALWAYS_KEEP_WEIGHT
            ))



def normalized_logit_diff_acdc_metric(data: ACDCEvalData, printing=False):
    B,V = data.patched.logits.size()

    # [batch_size]
    patched_correct = data.patched.correct[:,0]
    #print(data.unpatched.logits.size(), data.patched.logits.size(), data.corrupted.logits.size())
    A_logits_unpatched = data.unpatched.logits[torch.arange(B), patched_correct]
    A_logits_patched = data.patched.logits[torch.arange(B), patched_correct]
    A_logits_corrupted = data.corrupted.logits[torch.arange(B), patched_correct]

    corrupted_correct = data.corrupted.correct[:,0]
    B_logits_unpatched = data.unpatched.logits[torch.arange(B), corrupted_correct]
    B_logits_patched = data.patched.logits[torch.arange(B), corrupted_correct]
    B_logits_corrupted = data.corrupted.logits[torch.arange(B), corrupted_correct]

    # A and B are two potential outputs
    # if A patched > B patched, we are correct
    # else we are incorrect

    # thus we could just return A_logits_patched - B_logits_patched

    # however it is useful to "normalize" these values

    # in the worst case, our patching causes us to act like corrupted, and our diff will be
    # A_logits_corrupted - B_logits_corrupted
    # this will result in a small, negative value
    
    # in the best case, our patching will do nothing (cause us to act like unpatched), and our diff will be
    # A_logits_unpatched - B_logits_unpatched
    # this will result in a large, positive value

    # thus we can treat those as the "min" and "max" and normalize accordingly
    
    min_diff = A_logits_corrupted - B_logits_corrupted
    max_diff = A_logits_unpatched - B_logits_unpatched

    possible_range = (max_diff-min_diff)
    possible_range[possible_range == 0] = 1.0 # prevent divide by zero
    
    diff = A_logits_patched - B_logits_patched
    normalized_diff = (diff-min_diff)/torch.abs(possible_range) # abs prevents incorrect data from wanting to be more incorrect

    if printing:
        print(f"A corrupted {A_logits_corrupted}")
        print(f"B corrupted {B_logits_corrupted}")
        print(f"A unpatched {A_logits_unpatched}")
        print(f"B unpatched {B_logits_unpatched}")
        print(f"A patched {A_logits_patched}")
        print(f"B patched {B_logits_patched}")
        print(f"min diff {min_diff}")
        print(f"max diff {max_diff}")
        print(f"possible range {possible_range}")
        print(f"diff {diff}")
        print(f"normalized diff {normalized_diff}")
    # as described, 1.0 corresponds to acting like unpatched,
    # and 0.0 corresponds to acting like corrupted

    return torch.mean(normalized_diff)
import acdc
cfg = ACDCConfig(
    ckpt_directory = "blah",
    thresh = 0.0001,
    rollback_thresh = 0.0001,
    metric=acdc.accuracy_metric,
    # extra inference args
    model_kwargs=model_kwargs,
    # these are needed for doing graph pruning
    input_node=INPUT_NODE,
    output_node=OUTPUT_NODE,
    # batch size for evaluating data points
    batch_size=1,
    log_level=LOG_LEVEL_INFO,
    # if False, will be equivalent to batch_size=1
    batched = True,
    # set these two to false to use traditional ACDC
    # recursive will try patching multiple at a time (this is faster sometimes)
    recursive = True,
    # try_patching_multiple_at_same_time will evaluate many different patchings before commiting to any
    # and includes a rollback scheme if after patching one, the others get worse
    try_patching_multiple_at_same_time = True,
    ## if true, you metric will also have the logits from a run with no patching available
    # (useful for normalized logit diff)
    store_unpatched_logits = True,
)

# accuracy assumes only one name

clean_hooks(model)

print("getting unpatched")
from acdc import get_currently_patched_edge_hooks, eval_acdc, wrap_run_with_hooks, get_logits_of_predicted_next_token
unpatched_logits = get_logits_of_predicted_next_token(
        model=model,
        data=data.data,
        last_token_position=data.last_token_position,
        **model_kwargs
)

print("getting valid unpatched")
valid_unpatched_logits = get_logits_of_predicted_next_token(
        model=model,
        data=data.valid_data,
        last_token_position=data.valid_last_token_position,
        **model_kwargs
)

def eval_edges(edges_keeping, all_edges, valid=False):
    for edge in all_edges:
        edge.patching = False
        edge.checked = True
    for edge in edges_keeping:
        edge.patching = True
        edge.checked = True
    currently_patched_edge_hooks = get_currently_patched_edge_hooks(cfg=cfg, edges=edges)
    if valid:
        return eval_acdc(
                model=wrap_run_with_hooks(model=model, fwd_hooks=currently_patched_edge_hooks, **cfg.model_kwargs),
                data=data.valid_data,
                last_token_position=data.valid_last_token_position,
                correct=data.valid_correct,
                incorrect=data.valid_incorrect,
                metric=cfg.metric,
                num_edges=1,
                constrain_to_answers=data.constrain_to_answers,
                unpatched_logits=valid_unpatched_logits)[0].item()
    else:
        return eval_acdc(
            model=wrap_run_with_hooks(model=model, fwd_hooks=currently_patched_edge_hooks, **cfg.model_kwargs),
            data=data.data,
            last_token_position=data.last_token_position,
            correct=data.correct,
            incorrect=data.incorrect,
            metric=cfg.metric,
            num_edges=1,
            constrain_to_answers=data.constrain_to_answers,
            unpatched_logits=unpatched_logits)[0].item()

# can also do this instead to include negative contributions but I find the graph is larger, ymmv
#edges.sort(key=lambda x: -abs(x[0]))
print("sorting")
edges.sort(key=lambda edge: edge.score_diff_when_patched)

import math
def test_pos(pos):
    #print(f"testing pos {pos}")
    edges_to_keep = edges[:pos]
    metric = eval_edges(edges_to_keep, edges)
    #print(f"testing pos {pos} got metric {metric}")
    return metric

# from https://en.wikipedia.org/wiki/Binary_search_algorithm
def binary_search(n, T):
    L = 0
    R = n - 1
    while L != R:
        m = math.ceil((L + R) / 2)
        if test_pos(m) > T:
            R = m - 1
        else:
            L = m
    # go one further because this gives us below thresh
    return min(n-1, L+1)

print("check num always keep")
num_always_keep = 0
for edge in edges:
    if edge.score_diff_when_patched == ALWAYS_KEEP_WEIGHT:
        num_always_keep += 1
print(f"always keep {num_always_keep}")

ACC_THRESH = 0.85
torch.set_grad_enabled(False)
#cutoff = binary_search(len(edges), T=ACC_THRESH)

#edges_to_keep = edges[:cutoff]
#scores = [edge.score_diff_when_patched for edge in edges[:cutoff]]
#print(f"keeping top {cutoff} edges")
#metric = eval_edges(edges_to_keep, edges)
#print(f"got metric {metric}")    

#metric = eval_edges(edges_to_keep, edges, valid=True)
#print(f"got valid metric {metric}")    
# .85 has 3061 edges for 5 ITERS
# .85 has 2876 edges for 30 ITERS

# todo: fix the "only everything" bug for convs too
# patching format 0:
# 2162 edges (2016 always keep)

creating edges:
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
layer 34 pos 0 features 0
layer 34 pos 1 features 0
layer 34 pos 2 features 0
layer 34 pos 3 features 0
layer 34 pos 4 features 0
layer 34 pos 5 features 0
layer 34 pos 6 features 0
layer 34 pos 7 features 0
layer 34 pos 8 features 0
layer 34 pos 9 features 0
layer 34 pos 10 features 0
layer 34 pos 11 features 0
layer 34 pos 12 features 10
layer 34 pos 13 features 0
layer 34 pos 14 features 0
layer 34 pos 15 features 0
layer 34 pos 16 features 0
layer 34 pos 17 features 5081
35
36
37
38
39
40
41
42
43
44
45
46
47
getting unpatched


getting valid unpatched
sorting
check num always keep
always keep 864


In [ ]:

## just basic logit diff, doesn't go to zero

# occur more than 24 times, multiply by count
# testing pos 2112 got metric 0.8524999618530273
# keeping top 2112 edges

# 12 times
# testing pos 2178
# testing pos 2178 got metric 0.8499999642372131

# 30 times
# testing pos 1765 got metric 0.8399999737739563
# keeping top 1765 edges

## 0 to 1 logit diff, by corrective thing
# testing pos 5969 got metric 0.8474999666213989 for not multiply by count

# occur more than 6 times, multiply by count
# testing pos 2389 got metric 0.8524999618530273

# occur more than 12 times, multiply by count
# testing pos 2360 got metric 0.8499999642372131
# keeping top 2361 edges

# occur more than 24 times, multiply by count
# testing pos 2356 got metric 0.8524999618530273
# keeping top 2356 edges



layer 34 with 1215 unique features (0 duplicated)
  pos pos17,..., num sae 1215 min attr scaled -1.961
total num features 1215


In [7]:
## Checks

cutoffs = {}
minVal = 0.81
maxVal = 0.9424999952316284
for i in range(0,10):
    target = i/10.0
    targetAdjusted = (target*(maxVal-minVal)+minVal)
    print(f"target val {targetAdjusted}")
    cutoff = binary_search(len(edges), T=targetAdjusted)
    print(f"keeping {cutoff-num_always_keep} edges")
    edges_to_keep = edges[:cutoff]
    metric = eval_edges(edges_to_keep, edges, valid=True)
    print(f"got valid metric {metric}")    
    cutoffs[i] = cutoff
    sae_edges = defaultdict(lambda: defaultdict(lambda: []))
    counts = defaultdict(lambda: defaultdict(lambda: 0))
    position_map = {}
    toks = model.to_str_tokens(data.data[0])
    for l in range(L):
        position_map[l] = f'pos{l}{toks[l]}'
        
    for edge in edges_to_keep:
        if '.sae' in edge.output_node and not edge.label is None:
            # [pos:feature_i]
            label = edge.label[1:-1]
            pos, feature_i = label.split(":")
            pos = int(pos)
            if feature_i == 'KEEP': continue # dummy edge used to ensure sae always applied
            feature_i = int(feature_i)
            layer = int(edge.output_node.split(".")[0])
            attr = edge.score_diff_when_patched
            sae_edges[layer][pos].append((attr, feature_i))
            counts[layer][feature_i] += 1

    total_num_features = 0
    for layer in sorted(list(sae_edges.keys())):
        print(f"layer {layer} with {len(counts[layer])} unique features ({len([x for x in counts[layer].values() if x > 1])} duplicated)")
        total_num_features += len(counts[layer])
        values = sae_edges[layer]
        for pos in sorted(list(values.keys())):
            print(f"  pos {position_map[pos]} num sae {len(values[pos])} min attr scaled {'{:.3f}'.format(1000*min([x[0] for x in values[pos]]))}")
    print(f"total num features {total_num_features}")

target val 0.81
keeping 13 edges
got valid metric 0.8100000023841858
layer 34 with 13 unique features (0 duplicated)
  pos pos17,..., num sae 13 min attr scaled -1.961
total num features 13
target val 0.8232499995231629
keeping 257 edges
got valid metric 0.824999988079071
layer 34 with 257 unique features (0 duplicated)
  pos pos17,..., num sae 257 min attr scaled -1.961
total num features 257
target val 0.8364999990463258
keeping 661 edges
got valid metric 0.8374999761581421
layer 34 with 661 unique features (0 duplicated)
  pos pos17,..., num sae 661 min attr scaled -1.961
total num features 661
target val 0.8497499985694885
keeping 1122 edges
got valid metric 0.8499999642372131
layer 34 with 1122 unique features (0 duplicated)
  pos pos17,..., num sae 1122 min attr scaled -1.961
total num features 1122
target val 0.8629999980926514
keeping 1865 edges
got valid metric 0.8650000095367432
layer 34 with 1865 unique features (0 duplicated)
  pos pos17,..., num sae 1865 min attr scaled -1

KeyboardInterrupt: 

# Process Features

In [8]:
@dataclass
class SAEFeature:
    """Class for keeping track of an item in inventory."""
    layer: int
    pos: int
    feature_i: int
    attr: float
    original_attr: float
    original_count: float
    records: list = field(default_factory=lambda: [])

    def __repr__(self):
        return str(self.layer) + " " + str(self.pos) + " " + str(self.feature_i) + " " + str(self.attr)

    def __str__(self):
        return self.__repr__()

cutoff = 3407 + num_always_keep
edges_to_keep = edges[:cutoff]

sae_edges = defaultdict(lambda: defaultdict(lambda: []))
counts = defaultdict(lambda: defaultdict(lambda: 0))
position_map = {}
toks = model.to_str_tokens(data.data[0])
for l in range(L):
    position_map[l] = f'pos{l}{toks[l]}'
    
features = []
for edge in edges_to_keep:
    if '.sae' in edge.output_node and not edge.label is None:
        # [pos:feature_i]
        label = edge.label[1:-1]
        pos, feature_i = label.split(":")
        pos = int(pos)
        if feature_i == 'KEEP': continue # dummy edge used to ensure sae always applied
        feature_i = int(feature_i)
        layer = int(edge.output_node.split(".")[0])
        attr = edge.score_diff_when_patched
        features.append(SAEFeature(layer=layer, pos=pos, feature_i=feature_i, attr=attr, original_attr=edge.original_attr, original_count=edge.original_count))

# Get Max Activating Examples on a large dataset

(takes ~12 hours, just leave this running and come back later)

In [9]:

LAYER_STUDYING = 34

from datasets import load_dataset
from sae.data import chunk_and_tokenize
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
)
tokenizer = model.tokenizer
# too many processes crashes, probably memory issue
tokenized = chunk_and_tokenize(dataset, tokenizer, num_proc=8)

from dataclasses import dataclass, field
from functools import partial

def get_batched_index_into(indices):
    '''
    given data that is [B,N,V] and indicies that are [B,N,K] with each index being an index into the V space
    this gives you indexes you can use to access your values
    '''
    first_axis = []
    second_axis = []
    third_axis = []
    B, _, _ = indices.size()
    for b in range(B):
        second, third = get_index_into(indices[b])
        first_axis.append(torch.full(second.size(), fill_value=b, device=model.cfg.device))
        second_axis.append(second)
        third_axis.append(third)

    return torch.cat(first_axis), torch.cat(second_axis), torch.cat(third_axis)

def get_index_into(indices):
    '''
    given data that is [N,V] and indicies that are [N,K] with each index being an index into the V space
    this gives you indexes you can use to access your values
    '''
    num_data, num_per_data = indices.size()
    # we want
    # [0,0,0,...,] num per data of these
    # [1,1,1,...,] num per data of these
    # ...
    # [num_data-1, num_data-1, ...]
    first_axis_index = torch.arange(num_data, dtype=torch.long).view(num_data, 1)*torch.ones([num_data, num_per_data], dtype=torch.long)
    # now we flatten it so it has an index for each term aligned with our indices
    first_axis_index = first_axis_index.flatten()
    second_axis_index = indices.flatten()
    return first_axis_index, second_axis_index
global buffer
buffer = None
global features_by_layer
def sae_hook(
    x,
    hook,
    layer,
):
    global records_offset
    # s is [B,L,E]
    K = saes[layer].cfg.k
    sae = saes[layer]
    B,L,D = x.size()
    uncorrupted_features = sae.encode(x)
    top_acts, top_indices = uncorrupted_features.topk(K, sorted=False)
    global buffer
    if buffer is None:
        buffer = torch.zeros(uncorrupted_features.size(), device=model.cfg.device)
    buffer[:] = 0
    global features_by_layer
    # zero everything except the top k
    buffer[get_batched_index_into(top_indices)] = top_acts.flatten()
    for feature in features_by_layer[layer]:
        feature.records[records_offset:records_offset+B] = buffer[:,feature.pos,feature.feature_i]
    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    return sae_out
from tqdm import tqdm

def forward_check_features(data, features, batch_size):
    global records_offset
    
    global features_by_layer
    
    feat_is = set()
    for feature in features:
        feat_is.add(feature.feature_i)

    ACCUMULATE_FREQUENCY = 40

    #with open("layer_15_features_on_large_data.pkl", "rb") as f:
    #    features = pickle.load(f)
    features_by_layer = defaultdict(lambda: [])
    for feature in features:
        feature.records = torch.zeros([batch_size*ACCUMULATE_FREQUENCY], device=model.cfg.device)
        features_by_layer[feature.layer].append(feature)

    # only bother with SAE on the layers we are checking
    layers_to_apply_sae = sorted(list(features_by_layer.keys()))
    hooks = [(f'blocks.{layer}.hook_out_proj', partial(sae_hook, layer=layer)) for layer in layers_to_apply_sae]
    DATA_LEN = len(data)

    global records_offset
    global feature_offset
    records_offset = 0
    feature_offset = 0
    feature_i_top_k_indices = defaultdict(lambda: torch.tensor([], device=model.cfg.device))
    feature_i_top_k_values = defaultdict(lambda: torch.tensor([], device=model.cfg.device))
    K = 400
    ind = 0
    for batch_start in tqdm(list(range(0, DATA_LEN, batch_size))):
        batch_end = min(DATA_LEN, batch_start+batch_size)
        data_batch = data[batch_start:batch_end]['input_ids'][:,:128]
        _ = model.run_with_hooks(input=data_batch, fwd_hooks=hooks, fast_ssm=True, fast_conv=True)
        if ind % 100 == 0:
            print("saving")
            torch.save((dict(feature_i_top_k_indices), dict(feature_i_top_k_values)), "restopk.pkl")
            torch.save((dict(feature_i_top_k_indices), dict(feature_i_top_k_values)), "restopkb.pkl")
        ind += 1

        records_offset += batch_size

        if records_offset == batch_size*ACCUMULATE_FREQUENCY:
            records_offset = 0
            for feature in features:
                records = feature.records
                top = torch.topk(records, min(K, batch_size))
                # offset indices by total num seen so far
                top_indices = top.indices + feature_offset
                top_values = top.values
                merged_top_indices = torch.concatenate([feature_i_top_k_indices[feature.feature_i], top_indices])
                merged_top_values = torch.concatenate([feature_i_top_k_values[feature.feature_i], top_values])
                merged_top = torch.topk(merged_top_values, min(K, merged_top_indices.size()[0]))
                feature_i_top_k_indices[feature.feature_i] = merged_top_indices[merged_top.indices]
                feature_i_top_k_values[feature.feature_i] = merged_top_values[merged_top.indices]
                feature.records[:] = 0
            feature_offset += batch_size*ACCUMULATE_FREQUENCY

# we need to get their outputs on all positions, so make modified features
features_sorted_by_feat_i = defaultdict(lambda: [])
for feature in features:
    new_feats = []
    for pos in range(1,128):
        feat1 = SAEFeature(layer=LAYER_STUDYING, pos=pos, feature_i=feature.feature_i, attr=feature.attr, original_attr=feature.original_attr, original_count=feature.original_count)
        new_feats.append(feat1)
    features_sorted_by_feat_i[feature.feature_i] = new_feats

new_modified_feats = []
for f,feats in features_sorted_by_feat_i.items():
    new_modified_feats += feats

forward_check_features(tokenized, new_modified_feats, batch_size=100)

  0%|          | 1/5880 [00:06<10:11:35,  6.24s/it]

saving


  2%|▏         | 100/5880 [11:26<9:36:49,  5.99s/it]

saving


  3%|▎         | 200/5880 [23:43<30:49:02, 19.53s/it]

...

saving


 99%|█████████▊| 5800/5880 [12:17:59<25:44, 19.31s/it]

saving


100%|█████████▉| 5879/5880 [12:26:34<00:07,  7.62s/it]


RuntimeError: The expanded size of the tensor (60) must match the existing size (100) at non-singleton dimension 0.  Target sizes: [60].  Tensor sizes: [100]

## Move into big torch array for performance reasons

Also, gather feature activation values on max activating examples for positions other than the max activating position

In [14]:

with open("restopk.pkl", "rb") as f:
    feature_i_top_k_indices, feature_i_top_k_values = torch.load(f)

all_feature_i = sorted(list(feature_i_top_k_indices.keys()))
K = len(feature_i_top_k_values[all_feature_i[0]])
feature_to_storage_index = dict([(feat_i,index) for (index,feat_i) in enumerate(all_feature_i)])

token_data = torch.zeros(len(all_feature_i), K, 128, device=torch.device(model.cfg.device), dtype=torch.long)

torch.cuda.empty_cache()
import gc
gc.collect()
torch.cuda.empty_cache()

import tqdm
for feature_i in tqdm.tqdm(all_feature_i):
    storage_index = feature_to_storage_index[feature_i]
    indices = feature_i_top_k_indices[feature_i]
    for k, index in enumerate(indices):
        token_data[storage_index, k, :] = tokenized[torch.round(index).long().item()]['input_ids'][:128]

        
global features_by_layer
def sae_hook(
    x,
    hook,
    layer,
):
    # s is [B,L,E]
    K = saes[layer].cfg.k
    sae = saes[layer]
    B,L,D = x.size()
    uncorrupted_features = sae.encode(x)
    top_acts, top_indices = uncorrupted_features.topk(K, sorted=False)
    buffer = torch.zeros(uncorrupted_features.size(), device=model.cfg.device)
    global features_by_layer
    global records_offset
    # zero everything except the top k
    buffer[get_batched_index_into(top_indices)] = top_acts.flatten()
    for feature in features_by_layer[layer]:
            feature.records[records_offset:records_offset+B] = buffer[:,feature.pos,feature.feature_i]
    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    return sae_out

def forward_check_single_feature(data, feature, batch_size):
    global records_offset
    global features_by_layer

    TOTAL_SIZE = data.size()[0]

    features = []
    for pos in range(128):
        features.append(SAEFeature(layer=feature.layer, pos=pos, feature_i=feature.feature_i, attr=feature.attr, original_attr=feature.original_attr, original_count=feature.original_count))       

    features_by_layer = defaultdict(lambda: [])
    for feat in features:
        feat.records = torch.zeros([TOTAL_SIZE], device=model.cfg.device)
        features_by_layer[feat.layer].append(feat)

    # only bother with SAE on the layers we are checking
    layers_to_apply_sae = sorted(list(features_by_layer.keys()))
    hooks = [(f'blocks.{layer}.hook_out_proj', partial(sae_hook, layer=layer)) for layer in layers_to_apply_sae]
    DATA_LEN = len(data)

    global records_offset
    records_offset = 0
    for batch_start in list(range(0, DATA_LEN, batch_size)):
        batch_end = min(DATA_LEN, batch_start+batch_size)
        data_batch = data[batch_start:batch_end]
        _ = model.run_with_hooks(input=data_batch, fwd_hooks=hooks, fast_ssm=True, fast_conv=True)
        records_offset += batch_size
    return features

feature_act_data = torch.zeros(len(all_feature_i), K, 128, device=torch.device(model.cfg.device))

for feat_i in tqdm.tqdm(all_feature_i):
    feature = [feat for feat in features if feat.feature_i == feat_i][0]
    top_act_tokens = token_data[feature_to_storage_index[feat_i]]
    feature_acts = forward_check_single_feature(top_act_tokens, feature, batch_size=100)
    for i, f in enumerate(feature_acts):
        feature_act_data[feature_to_storage_index[feat_i], :, i] = f.records

torch.save((feature_act_data, token_data, feature_to_storage_index, features), "out_topk_data2.pkl")

  3%|▎         | 100/3407 [00:10<05:53,  9.34it/s]

# Load Processed Data (can be ran without running the above stuff once data is generated, though you do need to load SAEs first)

In [8]:
@dataclass
class SAEFeature:
    """Class for keeping track of an item in inventory."""
    layer: int
    pos: int
    feature_i: int
    attr: float
    original_attr: float
    original_count: float
    records: list = field(default_factory=lambda: [])

    def __repr__(self):
        return str(self.layer) + " " + str(self.pos) + " " + str(self.feature_i) + " " + str(self.attr)

    def __str__(self):
        return self.__repr__()

with open("out_topk_data.pkl", "rb") as f:
    (feature_act_data, token_data, feature_to_storage_index, features) = torch.load(f)

for feature in features:
    feature.records = []

features_sorted_by_feat_i = defaultdict(lambda: [])
for feature in features:
    features_sorted_by_feat_i[feature.feature_i].append(feature)

del feature_act_data
del token_data
torch.cuda.empty_cache()

# Manually test Feature

In [17]:
global features_by_layer
def sae_hook(
    x,
    hook,
    layer,
):
    # s is [B,L,E]
    K = saes[layer].cfg.k
    sae = saes[layer]
    B,L,D = x.size()
    uncorrupted_features = sae.encode(x)
    top_acts, top_indices = uncorrupted_features.topk(K, sorted=False)
    buffer = torch.zeros(uncorrupted_features.size(), device=model.cfg.device)
    global features_by_layer
    # zero everything except the top k
    buffer[get_batched_index_into(top_indices)] = top_acts.flatten()
    for feature in features_by_layer[layer]:
        if feature.pos < L: # sometimes prompt is too small to consider this feature
            feature.records += buffer[:,feature.pos,feature.feature_i].tolist()
    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    return sae_out

def forward_check_featuresf(data, features):
    
    global features_by_layer
    
    #with open("layer_15_features_on_large_data.pkl", "rb") as f:
    #    features = pickle.load(f)
    features_by_layer = defaultdict(lambda: [])
    for feature in features:
        feature.records = []
        features_by_layer[feature.layer].append(feature)

    # only bother with SAE on the layers we are checking
    layers_to_apply_sae = sorted(list(features_by_layer.keys()))
    hooks = [(f'blocks.{layer}.hook_out_proj', partial(sae_hook, layer=layer)) for layer in layers_to_apply_sae]
    _ = model.run_with_hooks(input=data, fwd_hooks=hooks, fast_ssm=True, fast_conv=True)
        
def clicked(arg):
    with outputTesting:
        clear_output()
        try:
            if eval_data.value.strip() == "":
                text = text_itemw.value
                tokenized_input = torch.tensor([model.tokenizer.bos_token_id] + model.tokenizer.encode(text), device=model.cfg.device).reshape(1,-1)
            else:
                tokenized_input = eval(eval_data.value).reshape(1, -1)
                print("eval to", tokenized_input)
            feature_i = int(feature_index.value)
            L = tokenized_input.size()[1]
            for feature in features:
                if feature.feature_i == feature_i:
                    feat = feature
            features_with_i = []
            for i in range(1, L):
                features_with_i.append(SAEFeature(layer=feat.layer, pos=i, feature_i=feature_i, attr=feat.attr, original_attr=feat.original_attr, original_count=feat.original_count))
            forward_check_featuresf(tokenized_input, features=features_with_i)
            activations = torch.zeros(L)
            for feature in features_with_i:
                activations[feature.pos] = feature.records[0]
            toks = model.to_str_tokens(tokenized_input[0])
            print(toks)
            token_pos = torch.argmax(activations).item()
            
            if activations[token_pos] == 0.0:
                token_pos = L
            out_toks = []
            print(activations)
            for j,tok in enumerate(toks):
                if len(tok.strip()) == 0:
                    tok = repr(tok)
                tok = tok.replace("\n", "\\n")
                colored = f"<span id='ayy'><font color='white'>{tok}</font></span>"
                if j < 1: continue
                if j == token_pos:
                    colored = f"<span id='ayy'><font color='red'>{tok}</font></span>"
                elif activations[j].item() > 0.01:
                    colored = f"<span id='ayy'><font color='pink'>{tok}</font></span>"
                if activations[j].item() > 0.01:
                    out_toks.append(f"{colored}{activations[j].item():.3f}")
                else:
                    out_toks.append(colored)
            simpler = model.tokenizer.decode(tokenized_input[0,1:token_pos+1])
            print(simpler)
            if token_pos == L:
                print("all zero")
            else:
                display(HTML(toks[token_pos] + "\t<br/><br/>\t" + "<span id='ayy'>" + simpler + "</span>\t<br/><br/>\t" + "".join(out_toks))) 
        except:
            print(traceback.format_exc())

                
text_itemw = widgets.Text(
    value='',
    placeholder='Test String',
    description='Test String',
    disabled=False,
    continuous_update=False,
)

feature_index = widgets.Text(
    value='',
    placeholder='Feature Index',
    description='Feature Index',
    disabled=False,
    continuous_update=False,
)
eval_data = widgets.Text(
    value='',
    placeholder='Eval Tokens',
    description='Eval Tokens',
    disabled=False,
    continuous_update=False,
)

test_button = widgets.Button(description = 'Test')   
test_button.on_click(clicked)

outputTesting = widgets.Output()

display(text_itemw)
display(feature_index)
display(eval_data)
display(test_button)
display(outputTesting)


Text(value='', continuous_update=False, description='Test String', placeholder='Test String')

Text(value='', continuous_update=False, description='Feature Index', placeholder='Feature Index')

Text(value='', continuous_update=False, description='Eval Tokens', placeholder='Eval Tokens')

Button(description='Test', style=ButtonStyle())

Output()

In [32]:
print(feature_act_data[feature_to_storage_index[2613]][0])

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], device='cuda:0')


In [18]:
"".join(['<|endoftext|>', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' proteins', ' Electron', 'rising', ' inaccur', ' invisible', ',...,', ' proteins', ' Electron', 'rising', ' inaccur', ' invisible', ',...,'])

'<|endoftext|> Electronrising inaccur invisible,..., proteins Electronrising inaccur invisible,..., proteins Electronrising inaccur invisible,...,'

In [19]:
with open("restopk.pkl", "rb") as f:
    feature_i_top_k_indices, feature_i_top_k_values = torch.load(f)




In [6]:

# token_data[feature_to_storage_index[2613],0]
# tokenized[int(feature_i_top_k_indices[2613][0].item())]['input_ids']
from datasets import load_dataset
from sae.data import chunk_and_tokenize
dataset = load_dataset(
    "togethercomputer/RedPajama-Data-1T-Sample",
    split="train",
    trust_remote_code=True,
)
tokenizer = model.tokenizer
# too many processes crashes, probably memory issue
tokenized = chunk_and_tokenize(dataset, tokenizer, num_proc=8)


In [30]:
tokenized[int(feature_i_top_k_indices[2613][0].item())]['input_ids']
tokenized[int(feature_i_top_k_indices[2613][0].item())]['input_ids']

tensor([  309,   858,   690,  ...,    13, 14635,  9137])

# Inspect TopK and Label

In [8]:
%%html
<style>
/*overwrite hard coded write background by vscode for ipywidges */
.cell-output-ipywidget-background {
   background-color: transparent !important;
}

/*set widget foreground text and color of interactive widget to vs dark theme color */
:root {
    --jp-widgets-color: var(--vscode-editor-foreground);
    --jp-widgets-font-size: var(--vscode-editor-font-size);
}
</style>

In [15]:

feature_labels_path = 'feature_labels_copy.pkl'
if not os.path.exists(feature_labels_path):
    print("no feature labels saved at" + feature_labels_path + " starting, fresh")
    feature_labels = {}
else:
    with open("feature_labels_copy.pkl", "rb") as f:
        feature_labels = pickle.load(f)

top_k_data = feature_act_data
global cur_feature_ind
cur_feature_ind = None
def display_unlabeled_feature():
    global cur_feature_ind
    global feature_labels
    #cur_feature_ind = 15112
    #text_item.value = feature_labels[cur_feature_ind]
    #display_feat(cur_feature_ind)
    available_features = features_sorted_by_feat_i.keys() - feature_labels.keys()
    maybe = []
    # sort by attr (they are negative, so this is largest first)
    available_features = sorted(list(available_features), key=lambda feat_i: -max([feat.original_count for feat in features_sorted_by_feat_i[feat_i]]))
    for f in available_features:
        print(f"feature {f}")
        for feat in features_sorted_by_feat_i[f]:
            print(f"pos {feat.pos} has attr {feat.original_attr} and count {feat.original_count}/{400*5}")
        cur_feature_ind = f
        if f in feature_labels:
            text_item.value = feature_labels[f]
        else:
            text_item.value = ""
        display_feat(cur_feature_ind)
        return

# provide fonts for rendering characters in other languages
display(HTML("""<link href='https://fonts.googleapis.com/css?family=Noto Sans' rel='stylesheet'>
<style>
/* Base Noto Sans and Serif for Latin, Greek, and Cyrillic */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans:wght@400;700&family=Noto+Serif:wght@400;700&display=swap');

/* East Asian scripts */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+JP:wght@400;700&family=Noto+Sans+KR:wght@400;700&family=Noto+Sans+SC:wght@400;700&family=Noto+Sans+TC:wght@400;700&display=swap');

/* South Asian scripts */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Devanagari:wght@400;700&family=Noto+Sans+Bengali:wght@400;700&family=Noto+Sans+Tamil:wght@400;700&display=swap');

/* Middle Eastern scripts */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Arabic:wght@400;700&family=Noto+Sans+Hebrew:wght@400;700&display=swap');

/* Other scripts */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Thai:wght@400;700&family=Noto+Sans+Ethiopic:wght@400;700&display=swap');

/* Specialty fonts */
@import url('https://fonts.googleapis.com/css2?family=Noto+Sans+Mono:wght@400;700&family=Noto+Color+Emoji&display=swap');

#ayy {
  font-family: 'Noto Sans', 'Noto Sans JP', 'Noto Sans KR', 'Noto Sans SC', 'Noto Sans TC', 
               'Noto Sans Devanagari', 'Noto Sans Bengali', 'Noto Sans Tamil', 
               'Noto Sans Arabic', 'Noto Sans Hebrew', 'Noto Sans Thai', 'Noto Sans Ethiopic',
               sans-serif;
}

/* Language-specific rules */
:lang(ja) { font-family: 'Noto Sans JP', sans-serif; }
:lang(ko) { font-family: 'Noto Sans KR', sans-serif; }
:lang(zh-CN) { font-family: 'Noto Sans SC', sans-serif; }
:lang(zh-TW) { font-family: 'Noto Sans TC', sans-serif; }
:lang(hi) { font-family: 'Noto Sans Devanagari', sans-serif; }
:lang(bn) { font-family: 'Noto Sans Bengali', sans-serif; }
:lang(ta) { font-family: 'Noto Sans Tamil', sans-serif; }
:lang(ar) { font-family: 'Noto Sans Arabic', sans-serif; }
:lang(he) { font-family: 'Noto Sans Hebrew', sans-serif; }
:lang(th) { font-family: 'Noto Sans Thai', sans-serif; }
:lang(am), :lang(ti) { font-family: 'Noto Sans Ethiopic', sans-serif; }

/* Emoji support */
.emoji {
  font-family: 'Noto Color Emoji', sans-serif;
}
</style>"""))


def display_feat(feature_i):
    covered_already = set()
    simpler_words = []
    for k in range(100):
        storage_index = feature_to_storage_index[feature_i]
        activations = top_k_data[storage_index, k]
        tokens = token_data[storage_index, k]
        token_pos = torch.argmax(activations).item()
        toks = model.to_str_tokens(tokens)
        relevant_str = "".join(toks[:token_pos+1])
        if relevant_str in covered_already:
            continue
        print(activations)
        covered_already.add(relevant_str)
        out_toks = []
        colors = [''] + ['red', 'orange', 'yellow', 'green']*256
        for j,tok in enumerate(toks):
            if len(tok.strip()) == 0:
                tok = repr(tok)
            tok = tok.replace("\n", "\\n")
            colored = f"<span id='ayy'><font color='white'>{tok}</font></span>"
            if j < 1: continue
            if j == token_pos:
                colored = f"<span id='ayy'><font color='red'>{tok}</font></span>"
            elif activations[j].item() > 0.01:
                colored = f"<span id='ayy'><font color='pink'>{tok}</font></span>"
            if activations[j].item() > 0.01:
                out_toks.append(f"{colored}{activations[j].item():.3f}")
            else:
                out_toks.append(colored)
        simpler = model.tokenizer.decode(tokens[1:token_pos+1])
        simpler_words.append(simpler.strip())

        display(HTML(toks[token_pos] + "\t||\t" + "<span id='ayy'>" + simpler + "</span>\t||\t" + "".join(out_toks)))
        '''
        if len(simpler_words) == 20:
            out_s = "<span id='ayy'>What do "
            for s in simpler_words:
                out_s += f'"{s.strip()}", '
            out_s += "have in common? Take a deep breath and think step by step."
            display(HTML(out_s + "</span>"))
        '''

def save_labels():
    with open("feature_labels_copy.pkl", "wb") as f:
        global feature_labelsb
        pickle.dump(feature_labelsb, f)
        print(f"done saving {len(feature_labelsb)}")

import traceback
out = widgets.Output()

def submitted(change):
    global cur_feature_ind
    global feature_labels
    if len(text_item.value.strip()) > 0 and (text_item.value != feature_labels[cur_feature_ind]):
        with out:
            try:
                times = 0
                clear_output()
                feature_labelsb[cur_feature_ind] = text_item.value
                save_labels()
                display_unlabeled_feature()
            except:
                print(traceback.format_exc())

text_item = widgets.Text(
    value='ffff',
    placeholder='Type something',
    description='String:',
    disabled=False,
    continuous_update=False,
)

display(text_item)
display(out)
text_item.observe(submitted, names='value')

with out:
    display_unlabeled_feature()

no feature labels saved atfeature_labels_copy.pkl starting, fresh


Text(value='ffff', continuous_update=False, description='String:', placeholder='Type something')

Output()

# Filter Away "no context" features

I found on IOI that many of the found features don't actually care about the context, and are just used to encode the current token. While useful, these aren't what we are looking for so we can discard them

In [ ]:


global buffer
buffer = None
def sae_hook(
    x,
    hook,
    layer,
):
    # s is [B,L,E]
    K = saes[layer].cfg.k
    sae = saes[layer]
    B,L,D = x.size()
    uncorrupted_features = sae.encode(x)
    top_acts, top_indices = uncorrupted_features.topk(K, sorted=False)
    buffer = torch.zeros(uncorrupted_features.size(), device=model.cfg.device)
    global features_by_layer
    # zero everything except the top k
    buffer[get_batched_index_into(top_indices)] = top_acts.flatten()
    for feature in features_by_layer[layer]:
        if feature.pos < L: # sometimes prompt is too small to consider this feature
            feature.records += buffer[:,feature.pos,feature.feature_i].tolist()
            feature.full_records += uncorrupted_features[:,feature.pos,feature.feature_i].tolist()
    # kernel can't handle doing all token positions at same time by default
    # but if we make it think B*L is a single batch index it works fine
    top_acts_flattened = top_acts.flatten(start_dim=0, end_dim=1)
    top_indices_flattened = top_indices.flatten(start_dim=0, end_dim=1)
    sae_out = sae.decode(top_acts_flattened, top_indices_flattened)
    sae_out = sae_out.unflatten(dim=0, sizes=(B,L))
    return sae_out

prompt = model.tokenizer.decode(data.data[0][1:])

prompt_tokens = model.to_tokens(prompt)
print(model.to_str_tokens(prompt_tokens))
logits_dla, activations_dla = model.run_with_cache(prompt_tokens)

features = 

features_sorted_by_feat_i = defaultdict(lambda: [])
for feature in features:
    features_sorted_by_feat_i[feature.feature_i].append(feature)


global features_by_layer
features_by_layer = defaultdict(lambda: [])
layers_to_apply_sae = [15]
feats = []
L = prompt_tokens.size()[1]
for layer in layers_to_apply_sae:
    for f, feats in features_sorted_by_feat_i.items():
        if len(feats) == 0:
            print("huh zero", f, feats)
            continue
        feat = feats[0]
        if feat.layer == layer:
            for i in range(1, L):
                features_by_layer[layer].append(SAEFeature(layer=feat.layer, pos=i, feature_i=feat.feature_i, attr=feat.attr))
                features_by_layer[layer][-1].records = []
                features_by_layer[layer][-1].full_records = []
hooks = [(f'blocks.{layer}.hook_out_proj', partial(sae_hook, layer=layer)) for layer in layers_to_apply_sae]
_ = model.run_with_hooks(input=prompt_tokens_dla, fwd_hooks=hooks, fast_ssm=True, fast_conv=True)


for layer in layers_to_apply_sae:
    # [B,L,E]
    x = activations_dla[f'blocks.{layer}.hook_ssm_input']
    # [B,L,E]
    y = activations_dla[f'blocks.{layer}.hook_y']
    # [B,L,E,N]
    A_bar = activations_dla[f'blocks.{layer}.hook_A_bar']
    # [B,L,E,N]
    B_bar = activations_dla[f'blocks.{layer}.hook_B_bar']
    # [B,L,E]
    skip = F.silu(activations_dla[f'blocks.{layer}.hook_skip'])
    # [B,L,N]
    C = activations_dla[f'blocks.{layer}.hook_C']
    # [D,E]
    W_out = model.blocks[layer].out_proj
    # [E]
    W_D = model.blocks[layer].W_D
    
    
    # [B,L,E,N]
    #h = torch.zeros([B,L,E,N], device=model.cfg.device)
    ys = []
    h = torch.zeros([B,E,N], device=model.cfg.device)
    for l in range(L):
        if l < 1:
            # [B,E,N]   [B,E,N]     [B,E,N]          [B,E,N]          [B,E]
            h        =    h    *  A_bar[:,l,:,:]  + B_bar[:,l,:,:] * x[:,l].view(B, E, 1) 
            h_0 = h
        else:       
            # [B,E,N]  [B,E,N]      [B,E,N] 
            h_0       =  h_0   *  A_bar[:,l,:,:] # do the A_bar multiply but ignore other x's
            # [B,E,N]   [B,E,N]     [B,E,N]          [B,E,N]          [B,E]
            h        =   h_0  + B_bar[:,l,:,:] * x[:,l].view(B, E, 1)
        # [B,E]    [B,E,N]       [B,N,1]   # this is like [E,N] x [N,1] for each batch

        y_l       =   h     @   C[:,l,:].view(B,N,1)
        # [B,E]              [B,E,1]
        y_l      =    y_l.view(B,E)
        ys.append(y_l)
    y = torch.stack(ys, dim=1)
    y_out         =     y +   x    *  W_D
    # [B,L,E]       [B,L,E]   [B,L,E]
    y_out         = y_out   *   skip
    # [B,L,D]       [B,L,E] x    [E,D]
    y_out         = y_out   @ W_out.weight.T

    sae = saes[layer]
    # F is feature size
    # [B,L,F]         [B,L,D]       [D]                [D,F]
    K = sae.cfg.k
    sae_vals = sae.encode(y_out)
    top_acts, top_indices = sae_vals.topk(K, sorted=False)
    buffer = torch.zeros(sae_vals.size(), device=model.cfg.device)
    # zero everything except the top k
    buffer[get_batched_index_into(top_indices)] = top_acts.flatten()
    #sae_vals      = relu((y_out - sae.b_dec) @ sae.encoder.weight.T)
    tests = 200
    t = 0
    for feat in sorted(list(features_by_layer[layer]), key=lambda f:  -abs(f.records[0] - buffer[0,f.pos,f.feature_i])):
        diff = buffer[0,feat.pos,feat.feature_i] - feat.records[0]
        if feat.pos == L-1:
            if abs(diff) > 0.001:
                print(diff.item(), feat.records[0], '->', buffer[0,l,feat.feature_i].item())
                print(feat.feature_i, feat.attr, dict(labs)[feat.feature_i])
                t += 1
        if t >= 200: break
    if t >= 200: break
    